# NOTEBOOK EJECUTIVO: RESUMEN COMPLETO
## MÓDULO 2: GESTIÓN DE CARTERAS
### Resumen integral de los temas 1, 2, 3 y 4 (hasta 4.3: Diseño de carteras con factores)

## ÍNDICE DEL DOCUMENTO

1. [Introducción](#introduccion)
2. [Tema 1: Introducción a Carteras](#tema1)
3. [Tema 2: Fundamentos de Gestión de Carteras y Riesgo](#tema2)
4. [Tema 3: CAPM y Modelo de Mercado](#tema3)
5. [Tema 4: Smart Betas y Modelos Multifactoriales](#tema4)
6. [Tabla Completa de Fórmulas](#tabla_formulas)
7. [Resumen Completo del Curso (Solo Texto)](#resumen_textual)
8. [Diagrama de Flujo del Curso](#diagrama)
9. [Conclusiones y Próximos Pasos](#conclusiones)

---

<a id='introduccion'></a>
## INTRODUCCIÓN

Este notebook ejecutivo presenta un resumen completo y detallado del Módulo 2: Gestión de Carteras, cubriendo desde los conceptos básicos de carteras de dos activos hasta los modelos multifactoriales y la construcción de carteras basadas en factores (TEMA 4: Smart Betas).

### Objetivos del Módulo:

* **Fundamentos Teóricos**: Profundizar en la teoría moderna de carteras
* **Aplicación Práctica**: Utilizar Python para simular, optimizar y validar estrategias de inversión

In [ ]:
# Imports necesarios para todo el notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configuración de visualización
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Librerías importadas correctamente")

<a id='tema1'></a>
## TEMA 1: INTRODUCCIÓN A CARTERAS

### Resumen de Conceptos Clave

| Sección | Tema | Conceptos Principales |
|---------|------|----------------------|
| **1.1** | Carteras de Dos Activos | Correlación (ρ), Diversificación, Fórmulas rentabilidad/varianza |
| **1.2** | Carteras de Cuatro Activos | Generalización a N activos, Matriz covarianza, Forma matricial |
| **1.3** | Análisis Diversificación | Riesgo sistemático, Límite cuando N→∞, Efecto diversificación |
| **Apéndice** | Fórmulas | Rentabilidad, Varianza, Covarianza, Contribución al riesgo |

### 1.1 Carteras de Dos Activos - Fórmulas Clave

**Rentabilidad Esperada:**
$$E(\tilde{R}_P) = w_1 E(\tilde{R}_1) + w_2 E(\tilde{R}_2)$$

**Varianza:**
$$\sigma_P^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\rho_{12}\sigma_1\sigma_2$$

**Casos de Correlación:**
- $\rho = 1$: No hay diversificación (línea recta)
- $\rho = -1$: Máxima diversificación (puede haber riesgo cero)
- $-1 < \rho < 1$: Diversificación parcial (curva)

In [ ]:
# Visualización: Efecto de la correlación en carteras de dos activos
def calc_ret_cartera(w1, w2, r1, r2):
    return w1 * r1 + w2 * r2

def calc_std_cartera(w1, w2, std1, std2, rho):
    return np.sqrt(w1**2 * std1**2 + w2**2 * std2**2 + 2 * w1 * w2 * std1 * std2 * rho)

# Parámetros de ejemplo
ret1, ret2 = 0.2, 0.1
std1, std2 = 0.25, 0.15
coef_corr_lst = [-1, -0.5, 0, 0.5, 1]
colors = ['#440154', '#3b528b', '#5ec962', '#fde725', '#ff0000']

fig, ax = plt.subplots(figsize=(14, 8))
x = np.linspace(0, 1, 101)

for idx, rho in enumerate(coef_corr_lst):
    ret_lst = []
    std_lst = []
    for w in x:
        reti = calc_ret_cartera(w, 1-w, ret1, ret2)
        stdi = calc_std_cartera(w, 1-w, std1, std2, rho)
        ret_lst.append(reti)
        std_lst.append(stdi)
    ax.plot(std_lst, ret_lst, 'o-', label=f'ρ = {rho}', color=colors[idx], markersize=4, alpha=0.7)

# Puntos de activos individuales
ax.scatter(std1, ret1, color='red', marker='*', s=300, label='Activo 1', zorder=5)
ax.scatter(std2, ret2, color='green', marker='*', s=300, label='Activo 2', zorder=5)

ax.set_xlabel('Volatilidad (Riesgo)', fontsize=12, fontweight='bold')
ax.set_ylabel('Retorno Esperado', fontsize=12, fontweight='bold')
ax.set_title('Efecto de la Correlación en Carteras de Dos Activos', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 1.3 Análisis del Efecto de la Diversificación - Resultado Clave

Para una cartera equiponderada ($w_i = 1/n$):

$$\sigma_p^2 = \frac{1}{n}\bar{V} + \left(1 - \frac{1}{n}\right)\bar{\sigma_{ij}}$$

**Cuando $n \to \infty$:**
$$\lim_{n \to \infty} \sigma_p^2 = \bar{\sigma_{ij}}$$

**Conclusión**: Solo queda el **riesgo sistemático** (riesgo de mercado) que no se puede eliminar con diversificación.

In [ ]:
# Visualización: Efecto de diversificación (riesgo vs número de activos)
def calcular_riesgo_diversificacion(n, var_media, cov_media):
    """Calcula el riesgo de una cartera equiponderada con n activos"""
    return np.sqrt((1/n) * var_media + (1 - 1/n) * cov_media)

# Parámetros de ejemplo
var_media = 0.04  # Varianza media de activos
cov_media = 0.01  # Covarianza media entre activos
riesgo_no_diversificable = np.sqrt(cov_media)

n_activos = np.arange(1, 101)
riesgos = [calcular_riesgo_diversificacion(n, var_media, cov_media) for n in n_activos]

fig, ax = plt.subplots(figsize=(12, 7))
ax.plot(n_activos, riesgos, 'b-', linewidth=2, label='Riesgo de la Cartera')
ax.axhline(y=riesgo_no_diversificable, color='r', linestyle='--', 
           linewidth=2, label='Riesgo Sistemático (Límite)')
ax.fill_between(n_activos, riesgos, riesgo_no_diversificable, 
                alpha=0.3, color='green', label='Riesgo Diversificable')

ax.set_xlabel('Número de Activos en la Cartera', fontsize=12, fontweight='bold')
ax.set_ylabel('Volatilidad de la Cartera', fontsize=12, fontweight='bold')
ax.set_title('Efecto de la Diversificación: Reducción del Riesgo', 
            fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

ax.annotate('Beneficios rápidos de diversificación', 
           xy=(10, riesgos[9]), xytext=(30, riesgos[9]+0.02),
           arrowprops=dict(arrowstyle='->', color='green', lw=2),
           fontsize=10, fontweight='bold')
ax.annotate('Riesgo sistemático no diversificable', 
           xy=(80, riesgo_no_diversificable), xytext=(50, riesgo_no_diversificable+0.03),
           arrowprops=dict(arrowstyle='->', color='red', lw=2),
           fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

<a id='tema2'></a>
##  TEMA 2: FUNDAMENTOS DE GESTIÓN DE CARTERAS Y RIESGO

### Resumen de Conceptos Clave

| Sección | Tema | Conceptos Principales |
|---------|------|----------------------|
| **2.1** | Optimización Media-Varianza | Función utilidad Markowitz, Cartera eficiente |
| **2.2** | Conceptos Optimización | Problema convexo, Solución global |
| **2.3** | CVXPY | Variables, Objetivo, Restricciones, Solución |
| **2.4** | Markowitz | Frontera eficiente, Coeficiente λ |
| **2.5** | Posiciones Cortas | Apalancamiento, Límites de endeudamiento |
| **2.6** | Teoría de Tobin | Activo libre de riesgo, CML |

### 2.1 Función de Utilidad del Inversor (Markowitz)

$$f(w) = w^T\mu - \lambda w^T\Sigma w$$

Donde:
- $w^T\mu$ = Rentabilidad esperada
- $\lambda w^T\Sigma w$ = Penalización por riesgo
- $\lambda$ = Coeficiente de aversión al riesgo

**Solución óptima:**
$$w^* = \frac{1}{2\lambda}\Sigma^{-1}\mu$$

In [ ]:
# Visualización conceptual: Frontera Eficiente y efecto de λ
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Simulación de frontera eficiente
sigma_range = np.linspace(0.05, 0.25, 100)
mu_efficient = 0.15 - 10 * (sigma_range - 0.15)**2 + 0.08

ax1.plot(sigma_range, mu_efficient, 'g-', linewidth=3, label='Frontera Eficiente')
ax1.fill_between(sigma_range, 0, mu_efficient, alpha=0.2, color='red', label='Región Ineficiente')
ax1.scatter(0.10, 0.12, color='blue', s=200, marker='*', label='Cartera Mínimo Riesgo', zorder=5)
ax1.scatter(0.20, 0.18, color='red', s=200, marker='*', label='Cartera Máxima Rentabilidad', zorder=5)
ax1.set_xlabel('Riesgo (Volatilidad)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Rentabilidad Esperada', fontsize=12, fontweight='bold')
ax1.set_title('Frontera Eficiente de Markowitz', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Gráfico de aversión al riesgo
lambda_vals = np.logspace(-2, 2, 100)
peso_riesgo = 1 / (1 + lambda_vals)
ax2.plot(lambda_vals, peso_riesgo, 'b-', linewidth=2)
ax2.fill_between(lambda_vals, 0, peso_riesgo, alpha=0.3, color='blue')
ax2.set_xlabel('Coeficiente de Aversión al Riesgo (λ)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Peso en Activos Riesgosos', fontsize=12, fontweight='bold')
ax2.set_title('Efecto de λ en la Composición de la Cartera', fontsize=13, fontweight='bold')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)
ax2.annotate('Alta tolerancia\\nal riesgo', xy=(0.01, 0.99), 
            xytext=(0.1, 0.8), arrowprops=dict(arrowstyle='->', color='green', lw=2),
            fontsize=10, fontweight='bold')
ax2.annotate('Baja tolerancia\\nal riesgo', xy=(100, 0.01), 
            xytext=(10, 0.2), arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

### 2.6 Teoría de Tobin - Activo Libre de Riesgo y CML

### 2.6 Teoría de Tobin - Activo Libre de Riesgo y CML

#### Características del Activo Libre de Riesgo ($R_f$):
- **Rentabilidad Fija**: $E(r_f) = r_f$
- **Varianza Cero**: $Var(r_f) = 0$
- **Covarianza Cero**: $Cov(r_f, r_i) = 0$

#### Cartera Mixta:
$$r_p = w r_f + (1-w) \mu$$
$$\sigma_p = (1-w) \sigma$$

**Capital Market Line (CML):**
$$\mu_p = r_f + \left(\frac{\mu_M - r_f}{\sigma_M}\right) \sigma_p$$

**Teorema de Separación de Tobin**: Todos los inversores deberían mantener la misma cartera de activos riesgosos (cartera de mercado M) y solo ajustar su asignación entre M y $R_f$.

In [ ]:
# Visualización: Teoría de Tobin - Activo Libre de Riesgo y CML
fig, ax = plt.subplots(figsize=(14, 8))

# Parámetros
rf = 0.03  # Tasa libre de riesgo anual
mu_M = 0.12  # Rentabilidad cartera de mercado
sigma_M = 0.15  # Volatilidad cartera de mercado

# Frontera eficiente de activos riesgosos (simulada)
sigma_frontier = np.linspace(0.05, 0.25, 100)
mu_frontier = 0.12 - 6 * (sigma_frontier - 0.15)**2 + 0.10
mu_frontier[mu_frontier < 0.08] = 0.08

# CML - Línea del Mercado de Capitales
sharpe_ratio = (mu_M - rf) / sigma_M
sigma_cml = np.linspace(0, 0.30, 100)
mu_cml = rf + sharpe_ratio * sigma_cml

# Graficar
ax.plot(sigma_frontier, mu_frontier, 'g-', linewidth=3, label='Frontera Eficiente (Activos Riesgosos)', alpha=0.7)
ax.plot(sigma_cml, mu_cml, 'r--', linewidth=3, label='Capital Market Line (CML)', alpha=0.8)
ax.scatter(0, rf, color='blue', s=300, marker='*', label='Activo Libre de Riesgo (Rf)', zorder=5)
ax.scatter(sigma_M, mu_M, color='red', s=300, marker='D', label='Cartera de Mercado (M)', zorder=5)

# Zonas de inversión
sigma_mezcla = np.linspace(0, sigma_M, 50)
mu_mezcla = rf + (mu_M - rf) / sigma_M * sigma_mezcla
ax.fill_between(sigma_mezcla, rf, mu_mezcla, alpha=0.2, color='green', label='Inversión Mixta')

sigma_apal = np.linspace(sigma_M, 0.30, 50)
mu_apal = rf + (mu_M - rf) / sigma_M * sigma_apal
ax.fill_between(sigma_apal, mu_frontier[len(mu_frontier)//2:], mu_apal, alpha=0.2, color='yellow', label='Apalancamiento')

ax.set_xlabel('Riesgo (Volatilidad)', fontsize=12, fontweight='bold')
ax.set_ylabel('Rentabilidad Esperada', fontsize=12, fontweight='bold')
ax.set_title('Teoría de Tobin: Activo Libre de Riesgo y Capital Market Line', 
            fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.01, 0.32)
ax.set_ylim(0, 0.25)

ax.annotate('0 < w < 1\\nInversión en Rf + M', 
           xy=(0.08, 0.06), fontsize=9, style='italic')
ax.annotate('w < 0\\nEndeudarse en Rf\\nInversión en M', 
           xy=(0.22, 0.18), fontsize=9, style='italic')

plt.tight_layout()
plt.show()

<a id='diagrama'></a>
##  DIAGRAMA DE FLUJO DEL CURSO

El siguiente diagrama muestra la estructura completa del curso y cómo cada tema se relaciona con los demás.

In [ ]:
# Diagrama de flujo completo del curso (ACTUALIZADO con TEMA 3)
fig, ax = plt.subplots(figsize=(20, 16))
ax.set_xlim(0, 10)
ax.set_ylim(0, 13)
ax.axis('off')

# Definir las cajas principales
cajas = [
    # Nivel superior - Título principal
    (5, 12, 'MÓDULO 2: GESTIÓN DE CARTERAS\nObjetivo: Teoría Moderna de Carteras + CAPM', '#1a237e', 3.2, 0.8),
    
    # Tema 1 - Rama izquierda
    (2, 10, 'TEMA 1\nIntroducción Carteras', '#1565c0', 1.8, 0.8),
    (2, 8.6, '1.1\nCarteras\n2 Activos', '#42a5f5', 1.4, 0.6),
    (2, 7.6, '1.2\nCarteras\n4 Activos', '#42a5f5', 1.4, 0.6),
    (2, 6.6, '1.3\nDiversificación', '#42a5f5', 1.4, 0.6),
    (2, 5.6, 'Apéndice\nFórmulas', '#90caf9', 1.4, 0.6),
    
    # Tema 2 - Rama central
    (5, 10, 'TEMA 2\nMarkowitz y Tobin', '#c62828', 1.8, 0.8),
    (5, 8.6, '2.1\nOptimización\nMedia-Varianza', '#ef5350', 1.4, 0.6),
    (5, 7.6, '2.2\nConceptos\nOptimización', '#ef5350', 1.4, 0.6),
    (5, 6.6, '2.3\nCVXPY', '#ef5350', 1.4, 0.6),
    (5, 5.6, '2.4\nMarkowitz', '#ef5350', 1.4, 0.6),
    (5, 4.6, '2.5\nPosiciones Cortas', '#ef5350', 1.4, 0.6),
    (5, 3.6, '2.6\nTeoría de Tobin', '#ef5350', 1.4, 0.6),
    
    # Tema 3 - Rama derecha (NUEVO)
    (8, 10, 'TEMA 3\nCAPM y Modelo\nMercado', '#f57c00', 1.8, 0.8),
    (8, 8.6, '3.2\nCAPM', '#ff9800', 1.4, 0.6),
    (8, 7.6, '3.3\nModelo de\nMercado', '#ff9800', 1.4, 0.6),
    (8, 6.6, '3.4\nEjercicio\nSP500', '#ffb74d', 1.4, 0.6),
    
    # Conceptos clave - Parte inferior
    (1, 1.5, 'CORRELACIÓN\nρ', '#81c784', 1, 0.4),
    (3, 1.5, 'DIVERSIFICACIÓN\nRiesgo', '#81c784', 1, 0.4),
    (5, 1.5, 'FRONTERA\nEFICIENTE\nCML', '#81c784', 1, 0.4),
    (7, 1.5, 'CAPM\nBeta\nSML', '#81c784', 1, 0.4),
    (9, 1.5, 'ALPHA\nRiesgo', '#81c784', 1, 0.4),
]

# Dibujar cajas
for x, y, text, color, width, height in cajas:
    box = FancyBboxPatch((x-width/2, y-height/2), width, height,
                        boxstyle="round,pad=0.05",
                        facecolor=color, edgecolor='black', linewidth=2, alpha=0.9)
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center',
           fontsize=7, fontweight='bold', color='white')

# Flechas principales
flechas_principales = [
    # De título a temas
    ((5, 11.6), (2.5, 10.4)),
    ((5, 11.6), (5, 10.4)),
    ((5, 11.6), (7.5, 10.4)),
    
    # Tema 1 - Flujo vertical
    ((2, 9.6), (2, 9.1)),
    ((2, 8.3), (2, 7.9)),
    ((2, 7.3), (2, 6.9)),
    ((2, 6.3), (2, 5.9)),
    
    # Tema 2 - Flujo vertical
    ((5, 9.6), (5, 9.1)),
    ((5, 8.3), (5, 7.9)),
    ((5, 7.3), (5, 6.9)),
    ((5, 6.3), (5, 5.9)),
    ((5, 5.3), (5, 4.9)),
    ((5, 4.3), (5, 3.9)),
    
    # Tema 3 - Flujo vertical (NUEVO)
    ((8, 9.6), (8, 9.1)),
    ((8, 8.3), (8, 7.9)),
    ((8, 7.3), (8, 6.9)),
    
    # Conexiones entre temas
    ((3.5, 5.5), (4.5, 5.5)),  # Tema 1 → Tema 2
    ((5.5, 3.5), (7.5, 8.5)),  # Tema 2.6 → Tema 3
    
    # A conceptos clave
    ((2, 5.3), (1, 1.9)),      # Diversificación → Correlación
    ((2, 6.3), (3, 1.9)),      # 1.3 → Diversificación
    ((5, 3.3), (5, 1.9)),      # Tobin → Frontera/CML
    ((8, 7.3), (7, 1.9)),      # CAPM → Beta/SML
    ((8, 6.3), (9, 1.9)),      # Modelo Mercado → Alpha/Riesgo
]

# Dibujar flechas
for start, end in flechas_principales:
    arrow = FancyArrowPatch(start, end,
                          arrowstyle='->', lw=2,
                          color='black', mutation_scale=20, alpha=0.8)
    ax.add_patch(arrow)

plt.title('DIAGRAMA DE FLUJO: ESTRUCTURA DEL CURSO DE GESTIÓN DE CARTERAS',
         fontsize=18, fontweight='bold', pad=30)
plt.tight_layout()
plt.show()

<a id='tabla_formulas'></a>
##  TABLA COMPLETA DE FÓRMULAS

Esta sección contiene **TODAS** las fórmulas del curso, organizadas por orden de presentación, con su utilidad y explicación breve.

In [ ]:
# Tabla completa de todas las fórmulas del curso
formulas_data = {
    'ID': range(1, 42),  # 41 fórmulas totales (35 anteriores + 6 nuevas de carteras multifactoriales)
    'Sección': [
        'Apéndice', 'Apéndice', 'Apéndice', 'Apéndice',  # 4
        '1.1', '1.1', '1.1',  # 3
        '1.2', '1.2', '1.2', '1.2',  # 4
        '1.3', '1.3',  # 2
        '2.1', '2.1', '2.1',  # 3
        '2.6', '2.6', '2.6', '2.6', '2.6', '2.6', '2.6', '2.6', '2.6', '2.6', '2.6', '2.6',  # 12
        '3.2', '3.2', '3.2', '3.3', '3.3', '3.3',  # 6 del CAPM
        '4.1',  # 1 de Fama-French
        '4.3', '4.3', '4.3', '4.3', '4.3', '4.3'  # 6 de carteras multifactoriales
    ],
    'Fórmula': [
        # Activos individuales
        r'$E(\tilde{R_i}) = \sum_{j=1}^{m} R_{ij} P_j$',
        r'$\sigma_i^2 = \sum_{j=1}^{m}(R_{ij}-E(\tilde{R_i}))^2 P_j$',
        r'$\sigma_i = \sqrt{\sigma_i^2}$',
        r'$SV_i = \sum_{j=1}^{m}(R_{ij}^*-E(\tilde{R_i}))^2 P_j$',
        
        # Carteras 2 activos
        r'$E(\tilde{R_P}) = w_1 E(\tilde{R_1}) + w_2 E(\tilde{R_2})$',
        r'$\sigma_P^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\rho_{12}\sigma_1\sigma_2$',
        r'$\sigma_P = \sqrt{\sigma_P^2}$',
        
        # Carteras N activos
        r'$E(\tilde{R_p}) = \sum_{i=1}^{N}w_i E(\tilde{R_i})$',
        r'$\sigma_p^2 = \sum_{i=1}^{N}w_i^2\sigma_i^2 + \sum_{i=1}^{N}\sum_{j=1}^{N}w_iw_j\sigma_{ij}$',
        r'$\sigma_p^2 = \mathbf{w}^T\mathbf{C}\mathbf{w}$',
        r'$\rho_{ij} = \frac{\sigma_{ij}}{\sigma_i\sigma_j}$',
        
        # Diversificación
        r'$\sigma_p^2 = \frac{1}{n}\bar{V} + \left(1-\frac{1}{n}\right)\bar{\sigma_{ij}}$',
        r'$\lim_{n \to \infty} \sigma_p^2 = \bar{\sigma_{ij}}$',
        
        # Optimización Markowitz
        r'$\max f(w) = w^T\mu - \lambda w^T\Sigma w$',
        r'$\frac{\partial f(w)}{\partial w} = \mu - 2\lambda\Sigma w = 0$',
        r'$w^* = \frac{1}{2\lambda}\Sigma^{-1}\mu$',
        
        # Tobin - Activo libre de riesgo
        r'$E(r_f) = r_f$',
        r'$Var(r_f) = 0$',
        r'$Cov(r_f, r_i) = 0$',
        r'$r_p = w r_f + (1-w) \mu$',
        r'$\sigma_p = (1-w)\sigma$',
        r'$w = \frac{r_p - \mu}{r_f - \mu}$',
        r'$w = 1 - \frac{\sigma_p}{\sigma}$',
        
        # Tobin - CML
        r'$\max w_0r_f + \mu^T w - \lambda w^T\Sigma w$',
        r'$\pi = \mu - r_f$',
        r'$\pi - 2\lambda\Sigma w = 0$',
        r'$w^* = \frac{1}{\lambda}\Sigma^{-1}\pi$',
        r'$\mu_p = r_f + \left(\frac{\mu_M - r_f}{\sigma_M}\right)\sigma_p$',
        
        # CAPM y Modelo de Mercado (TEMA 3)
        r'$E[\tilde{r}_i]-r_f=\beta_i(E[\tilde{r}_M-r_f])$',
        r'$\beta_i=\frac{Cov(\tilde{r_i},\tilde{r_M})}{\sigma^2_M}$',
        r'$\mu_p=r_f+\beta_p (\mu_M-r_f)$',
        r'$R_{i,t} - R_f = \alpha_i + \beta_i (R_{M,t} - R_f) + \varepsilon_{i,t}$',
        r'$\sigma^2(\pi_i)=\beta_i^2\sigma^2(\pi_M)+\sigma^2(\varepsilon_i)$',
        r'$\beta_p=\frac{\sigma_{pM}}{\sigma^2_M}$',
        
        # Modelo de Fama-French (TEMA 4.1)
        r'$E(r_i) = r_f + \beta_{i,MKT} E(r_m - r_f) + \beta_{i,SMB} E(SMB) + \beta_{i,HML} E(HML) + \beta_{i,MOM} E(MOM)$',
        
        # Carteras Multifactoriales (TEMA 4.3)
        r'$MOM_{i,t} = \prod_{j=2}^{12}(1+r_{i,t-j})-1$',
        r'$z_{i,k} = \frac{señal_{i,k}-\mu_k}{\sigma_k}$',
        r'$b(w) = X^T w \in \mathbb{R}^K$',
        r'$(X^T w)_k = \sum_{i=1}^N w_i X_{i,k}$',
        r'$\|X^T w - b^*\|^2_W = \sum_{k=1}^K W_k ((X^T w)_k - b^*_k)^2$',
        r'$W_k = \frac{1}{\sigma^2_k}$'
    ],
    'Nombre/Descripción': [
        'Rentabilidad Esperada de un Activo',
        'Varianza de un Activo',
        'Desviación Típica (Volatilidad)',
        'Semivarianza de un Activo',
        'Rentabilidad Cartera 2 Activos',
        'Varianza Cartera 2 Activos',
        'Volatilidad Cartera 2 Activos',
        'Rentabilidad Cartera N Activos',
        'Varianza Cartera N Activos (Expansión)',
        'Varianza Cartera N Activos (Matricial)',
        'Coeficiente de Correlación',
        'Varianza Cartera Equiponderada',
        'Límite Diversificación (Riesgo Sistemático)',
        'Función Utilidad Markowitz',
        'Condición Primer Orden (FOC)',
        'Pesos Óptimos Markowitz',
        'Rentabilidad Esperada Activo Libre Riesgo',
        'Varianza Activo Libre Riesgo',
        'Covarianza Activo Libre Riesgo',
        'Rentabilidad Cartera Mixta',
        'Volatilidad Cartera Mixta',
        'Peso Rf (Objetivo Rentabilidad)',
        'Peso Rf (Objetivo Volatilidad)',
        'Función Utilidad con Rf',
        'Premio al Riesgo',
        'FOC con Activo Libre Riesgo',
        'Pesos Óptimos Cartera Mixta',
        'Capital Market Line (CML)',
        'Capital Asset Pricing Model (CAPM)',
        'Beta de un Activo',
        'Security Market Line (SML)',
        'Modelo de Mercado',
        'Descomposición Riesgo (Sistemático + Específico)',
        'Beta de una Cartera',
        'Modelo de Fama-French de 4 Factores',
        'Momentum 12-2',
        'Normalización de Señal de Factor (Z-score)',
        'Exposición de Cartera a Factores',
        'Exposición de Cartera al Factor k',
        'Tracking de Exposiciones',
        'Peso de Exposición por Factor'
    ],
    'Utilidad': [
        'Calcular rendimiento promedio esperado de un activo',
        'Medir dispersión/dispersión de rendimientos',
        'Medir riesgo en unidades de rentabilidad',
        'Medir riesgo solo de pérdidas (abajo de la media)',
        'Calcular rentabilidad esperada de cartera simple',
        'Calcular riesgo total incluyendo correlación',
        'Expresar riesgo en unidades interpretables',
        'Generalizar rentabilidad a múltiples activos',
        'Calcular riesgo considerando todas las covarianzas',
        'Forma compacta y computacionalmente eficiente',
        'Medir grado de relación lineal entre activos',
        'Analizar efecto diversificación en cartera equitativa',
        'Identificar riesgo no diversificable (sistemático)',
        'Maximizar utilidad balanceando riesgo-rendimiento',
        'Encontrar máximo de función objetivo',
        'Obtener cartera óptima para aversión al riesgo λ',
        'Caracterizar activo sin riesgo',
        'Confirmar ausencia de variabilidad en Rf',
        'Confirmar independencia de Rf con activos riesgosos',
        'Calcular rentabilidad cartera mezclando Rf y riesgoso',
        'Calcular riesgo de cartera mixta',
        'Determinar pesos para alcanzar rentabilidad objetivo',
        'Determinar pesos para alcanzar riesgo objetivo',
        'Optimizar cartera incluyendo activo libre de riesgo',
        'Medir compensación por asumir riesgo',
        'Condición óptima con Rf disponible',
        'Pesos óptimos de activos riesgosos con Rf',
        'Relación lineal riesgo-rendimiento en carteras eficientes',
        'Relación entre premio activo y premio mercado según riesgo sistemático',
        'Medir sensibilidad de activo a movimientos del mercado',
        'Relación rentabilidad-beta para cualquier activo o cartera',
        'Modelo de regresión para estimar alpha y beta empíricamente',
        'Separar riesgo total en componente sistemático (beta²*σ²M) y específico (σ²ε)',
        'Beta de cartera como ponderación de betas individuales',
        'Extender CAPM incorporando factores de tamaño (SMB), valor (HML) y momentum (MOM)',
        'Calcular momentum como rendimientos acumulados de t-12 a t-2 (evitando reversión a corto plazo)',
        'Estandarizar señales de factores para comparabilidad cross-sectional (z-score)',
        'Calcular exposición agregada de cartera a factores mediante producto matricial X^T w',
        'Calcular exposición de cartera a factor específico k como suma ponderada de características',
        'Minimizar desviación de exposiciones objetivo ponderada por importancia de factores',
        'Determinar peso de exposición normalizando por incertidumbre o tolerancia al error'
    ],
    'Explicación Breve': [
        'Suma ponderada de todos los posibles rendimientos por sus probabilidades',
        'Promedio de desviaciones al cuadrado respecto a la media',
        'Raíz cuadrada de la varianza; medida estándar de riesgo',
        'Solo considera desviaciones negativas; medida de riesgo downside',
        'Promedio ponderado simple; NO depende de correlación',
        'Incluye términos de varianza individual y covarianza (correlación)',
        'Desviación estándar; medida directa de volatilidad',
        'Suma ponderada de rentabilidades individuales',
        'Incluye todos los términos de varianza y covarianza cruzada',
        'Multiplicación matricial: w^T * C * w donde C es matriz covarianza',
        'Normaliza covarianza entre -1 y 1; mide relación lineal',
        'Riesgo depende del número de activos n; combina varianza y covarianza media',
        'Al aumentar activos, solo queda covarianza media (riesgo sistemático)',
        'Balance entre rentabilidad (positivo) y riesgo penalizado por λ',
        'Derivada igualada a cero para encontrar máximo',
        'Solución analítica; inversa de Σ es clave computacional',
        'Rendimiento conocido y fijo (constante)',
        'No hay variabilidad en rendimientos',
        'No hay relación estadística con otros activos',
        'Combinación lineal simple de Rf y activo riesgoso',
        'Riesgo depende solo del componente riesgoso (proporción 1-w)',
        'Despejar peso w cuando se conoce rentabilidad objetivo r_p',
        'Despejar peso w cuando se conoce volatilidad objetivo σ_p',
        'Extensión de Markowitz incluyendo activo libre de riesgo',
        'Diferencia entre rentabilidad esperada y tasa libre de riesgo',
        'Condición de primer orden en problema con Rf',
        'Pesos óptimos usando premio al riesgo π',
        'Ecuación de línea recta: todas las carteras eficientes están en esta línea',
        'El CAPM relaciona el premio de un activo con su beta y el premio de mercado',
        'Beta mide cuánto varía el activo por cada variación unitaria del mercado',
        'SML extiende CAPM a cualquier activo o cartera (no solo eficientes)',
        'Regresión empírica que permite estimar alpha (sobre/sub-valoración) y beta',
        'El riesgo total se descompone en riesgo sistemático (beta²*σ²M) más riesgo específico (σ²ε)',
        'El beta de una cartera es la covarianza con mercado dividida por varianza de mercado',
        'Modelo multifactorial que explica rendimientos mediante factores de mercado, tamaño, valor y momentum',
        'Momentum 12-2 captura persistencia de rendimientos pasados excluyendo mes t-1 y t para evitar reversión',
        'Z-score normaliza señales cross-sectionalmente permitiendo comparación entre activos en cada período',
        'La exposición de cartera es el producto matricial de matriz características X por vector pesos w',
        'Cada exposición factorial es suma ponderada de características individuales de activos',
        'Tracking error cuadrático ponderado mide desviación de exposiciones objetivo',
        'Peso inverso a varianza normaliza factores por incertidumbre o especifica tolerancia explícita'
    ]
}

df_formulas = pd.DataFrame(formulas_data)

# Crear tabla visual mejorada
fig, ax = plt.subplots(figsize=(20, 35))
ax.axis('tight')
ax.axis('off')

# Crear tabla con todas las columnas
table_data = []
headers = ['ID', 'Sección', 'Fórmula', 'Nombre', 'Utilidad', 'Explicación']

for idx, row in df_formulas.iterrows():
    # Truncar explicaciones muy largas para visualización
    explicacion = row['Explicación Breve'][:80] + '...' if len(row['Explicación Breve']) > 80 else row['Explicación Breve']
    utilidad = row['Utilidad'][:70] + '...' if len(row['Utilidad']) > 70 else row['Utilidad']
    
    table_data.append([
        str(row['ID']),
        row['Sección'],
        row['Fórmula'],
        row['Nombre/Descripción'],
        utilidad,
        explicacion
    ])

table = ax.table(cellText=table_data, colLabels=headers,
                cellLoc='left', loc='center', bbox=[0, 0, 1, 1])

table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1, 1.8)

# Estilo de encabezados
for i in range(len(headers)):
    table[(0, i)].set_facecolor('#1a237e')
    table[(0, i)].set_text_props(weight='bold', color='white', fontsize=9)

# Alternar colores de filas
for i in range(1, len(table_data) + 1):
    for j in range(len(headers)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#f5f5f5')
        else:
            table[(i, j)].set_facecolor('#ffffff')
        
        # Fórmulas en negrita y color azul
        if j == 2:  # Columna de fórmulas
            table[(i, j)].set_text_props(weight='bold', color='#0d47a1', fontsize=9)

plt.title('TABLA COMPLETA DE FÓRMULAS: MÓDULO 2 - GESTIÓN DE CARTERAS', 
         fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print(f"\nTotal de fórmulas documentadas: {len(df_formulas)}")

### Resumen por Secciones:

**Apéndice (4 fórmulas):**
- Fórmulas básicas de activos individuales (rentabilidad, varianza, desviación típica, semivarianza)

**Tema 1.1 - Carteras de Dos Activos (3 fórmulas):**
- Rentabilidad esperada
- Varianza (con correlación)
- Volatilidad

**Tema 1.2 - Carteras de N Activos (4 fórmulas):**
- Generalización a N activos
- Forma matricial
- Coeficiente de correlación

**Tema 1.3 - Diversificación (2 fórmulas):**
- Varianza de cartera equiponderada
- Límite cuando n→∞ (riesgo sistemático)

**Tema 2.1 - Optimización Markowitz (3 fórmulas):**
- Función de utilidad
- Condición de primer orden
- Solución óptima

**Tema 2.6 - Teoría de Tobin (12 fórmulas):**
- Características activo libre de riesgo (3)
- Carteras mixtas (4)
- Capital Market Line (CML) (5)

**Tema 3.2 - CAPM (3 fórmulas):**
- Fórmula del CAPM
- Coeficiente Beta
- Security Market Line (SML)

**Tema 3.3 - Modelo de Mercado (3 fórmulas):**
- Modelo de mercado (regresión)
- Descomposición riesgo sistemático/específico
- Beta de cartera

**Tema 4.1 - Modelo de Fama-French (1 fórmula):**
- Modelo de Fama-French de 4 factores (con momentum)

**Tema 4.3 - Carteras Multifactoriales (6 fórmulas):**
- Momentum 12-2
- Normalización de señal de factor (z-score)
- Exposición de cartera a factores
- Exposición de cartera al factor k
- Tracking de exposiciones
- Peso de exposición por factor

**Total: 41 fórmulas principales**

---

### Nota sobre Implementación:

Muchas de estas fórmulas se implementan en Python usando:
- `numpy` para operaciones matriciales
- `pandas` para manejo de datos
- `cvxpy` para optimización
- Las fórmulas matriciales son especialmente eficientes para carteras con muchos activos

<a id='resumen_textual'></a>
## RESUMEN COMPLETO DEL CURSO (SOLO TEXTO)

Este resumen presenta una descripción detallada y completa de todos los conceptos, teorías y aplicaciones del Módulo 2: Gestión de Carteras, desde los fundamentos básicos de carteras de dos activos hasta los modelos multifactoriales de Fama-French (Smart Betas).

---

### **TEMA 1: INTRODUCCIÓN A CARTERAS**

#### **1.1 Carteras de Dos Activos**

Una cartera de activos define una colección de activos financieros, como acciones, bonos y otros instrumentos de inversión. La composición de la cartera se determina con el objetivo de alcanzar una meta financiera específica, como maximizar el rendimiento, minimizar el riesgo o encontrar cierto equilibrio entre ambos.

En el caso más simple de dos activos, definimos:
- **Ponderaciones**: $w_1$ es la proporción de inversión en el activo 1, y $w_2 = 1 - w_1$ es la proporción en el activo 2. Estas ponderaciones deben sumar 1.0 (representando el 100% de la inversión).
- **Rentabilidad Esperada**: La rentabilidad esperada de la cartera es una suma ponderada de las rentabilidades esperadas de cada activo. Es decir, $E(\tilde{R}_P) = w_1 E(\tilde{R}_1) + w_2 E(\tilde{R}_2)$. Esta relación es importante porque no depende de la correlación entre los activos.
- **Varianza**: La varianza de la cartera, que mide el riesgo, incluye no solo las varianzas individuales de cada activo, sino también la relación entre ellos a través de la correlación. La fórmula es: $\sigma_P^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\rho_{12}\sigma_1\sigma_2$, donde $\rho_{12}$ es el coeficiente de correlación entre los activos.

**El papel crucial de la correlación**: El coeficiente de correlación ($\rho_{12}$) determina el beneficio de la diversificación:

- **Correlación Perfecta Positiva ($\rho = 1$)**: Cuando dos activos están perfectamente correlacionados positivamente, no hay beneficios de diversificación. La volatilidad de la cartera es simplemente la suma ponderada de las volatilidades individuales. La cartera se comporta como una combinación lineal simple, representada por una línea recta en el espacio riesgo-retorno.

- **Correlación Perfecta Negativa ($\rho = -1$)**: Este es el caso de máxima diversificación. Con correlación perfecta negativa, es posible crear una cartera con riesgo cero (o muy bajo) combinando los activos en las proporciones correctas. La cartera puede estar representada por dos líneas rectas que se unen en un punto de riesgo mínimo.

- **Correlación Intermedia ($-1 < \rho < 1$)**: En la práctica, la mayoría de las correlaciones están entre estos extremos. La diversificación parcial reduce el riesgo sin eliminarlo completamente. En este caso, la relación riesgo-retorno se representa por una curva convexa en el espacio riesgo-retorno.

La volatilidad de la cartera se calcula como la raíz cuadrada de la varianza: $\sigma_P = \sqrt{\sigma_P^2}$.

#### **1.2 Carteras de Cuatro Activos y Generalización a N Activos**

La generalización de dos a cuatro activos, y luego a N activos, introduce conceptos importantes:

**Rentabilidad de una Cartera de N Activos**: La rentabilidad esperada se generaliza como $E(\tilde{R}_p) = \sum_{i=1}^{N}w_i E(\tilde{R}_i)$, donde $w_i$ representa la ponderación del activo $i$ en la cartera. Esta es una suma ponderada simple y no depende de las relaciones entre activos.

**Varianza de una Cartera de N Activos**: La varianza se vuelve más compleja porque debe considerar todas las posibles relaciones (covarianzas) entre pares de activos. La fórmula en su forma expandida es:

$\sigma_p^2 = \sum_{i=1}^{N} w_i^2\sigma_i^2 + \sum_{i=1}^{N}\sum_{j=1}^{N}w_iw_j\sigma_{ij}$ (con $i \neq j$)

Donde el primer término representa las varianzas individuales de cada activo, y el segundo término captura todas las covarianzas (relaciones) entre pares de activos.

**Forma Matricial**: Una forma más compacta y computacionalmente eficiente es la representación matricial: $\sigma_p^2 = \mathbf{w}^T\mathbf{C}\mathbf{w}$, donde $\mathbf{w}$ es el vector de ponderaciones y $\mathbf{C}$ es la matriz de covarianzas. Esta forma es esencial para cálculos eficientes con múltiples activos.

**Coeficiente de Correlación**: La correlación entre dos activos se calcula como $\rho_{ij} = \frac{\sigma_{ij}}{\sigma_i\sigma_j}$, normalizando la covarianza para que esté siempre entre -1 y 1. Esta medida indica el grado de relación lineal entre los activos.

**Contribución al Riesgo y Rendimiento**: Es importante entender cómo cada activo contribuye al riesgo y rendimiento total de la cartera:
- **Contribución al Rendimiento**: $w_i E(\tilde{R}_i)$ - depende solo del peso del activo.
- **Contribución al Riesgo**: $w_i^2 cov(\tilde{R}_i, \tilde{R}_P)$ - depende tanto del peso como de la covarianza con la cartera.

Un activo con rentabilidad positiva y covarianza negativa con la cartera aumentará el rendimiento mientras disminuye el riesgo, lo cual es altamente deseable.

#### **1.3 Análisis del Efecto de la Diversificación**

El análisis de diversificación revela un resultado fundamental en teoría de carteras. Para una cartera equiponderada (donde cada activo tiene peso $w_i = 1/n$), la varianza puede expresarse como:

$\sigma_p^2 = \frac{1}{n}\bar{V} + \left(1 - \frac{1}{n}\right)\bar{\sigma_{ij}}$

Donde $\bar{V}$ es la varianza media de los activos individuales y $\bar{\sigma_{ij}}$ es la covarianza media entre activos.

**El Límite de la Diversificación**: Cuando el número de activos $n$ tiende a infinito, se obtiene:

$\lim_{n \to \infty} \sigma_p^2 = \bar{\sigma_{ij}}$

**Riesgo Sistemático vs. Riesgo Específico**: Este resultado tiene implicaciones profundas:

- **Riesgo Específico (Diversificable)**: Es la parte del riesgo que puede eliminarse mediante la diversificación. Viene representada por el término $\frac{1}{n}\bar{V}$, que disminuye a medida que aumenta el número de activos.

- **Riesgo Sistemático (No Diversificable)**: Es el riesgo que permanece incluso con diversificación completa. Viene representado por $\bar{\sigma_{ij}}$, la covarianza media entre activos, que refleja factores comunes que afectan a todos los activos del mercado.

Este concepto es fundamental: la diversificación puede eliminar el riesgo específico de cada activo, pero nunca puede eliminar el riesgo sistemático inherente al mercado en su conjunto. Este resultado justifica teóricamente por qué los inversores exigen un premio por asumir riesgo, ya que este riesgo sistemático no puede evitarse.

---

### **TEMA 2: FUNDAMENTOS DE GESTIÓN DE CARTERAS Y RIESGO**

#### **2.1 Introducción a la Optimización Media-Varianza**

Hasta este punto, hemos analizado carteras aleatorias o simples combinaciones de activos. La pregunta clave es: **¿cuáles son las carteras eficientes?** Una cartera es eficiente si maximiza el rendimiento para un nivel de riesgo dado, o minimiza el riesgo para un nivel de rendimiento dado.

**El Problema de la Elección Óptima**: Dada una nube de posibles carteras en el espacio riesgo-retorno, la optimización media-varianza busca identificar aquellas carteras que están en la "frontera eficiente" - el conjunto de carteras que ofrecen el mejor equilibrio posible entre riesgo y rendimiento.

**La Función de Utilidad del Inversor**: Harry Markowitz, premio Nobel de Economía, propuso que los inversores racionales buscan maximizar su utilidad esperada. Bajo ciertas hipótesis (función de utilidad cuadrática o rendimientos normalmente distribuidos), esta maximización de utilidad se convierte en un problema de optimización media-varianza.

**La Función Objetivo de Markowitz**: Markowitz propone maximizar la siguiente función de utilidad:

$\max f(w) = w^T\mu - \lambda w^T\Sigma w$

Donde:
- $w^T\mu$ representa la rentabilidad esperada de la cartera (beneficio).
- $\lambda w^T\Sigma w$ representa la penalización por riesgo (coste).
- $\lambda$ es el coeficiente de aversión al riesgo del inversor.

**Interpretación del Coeficiente de Aversión al Riesgo ($\lambda$)**:
- Un $\lambda$ **alto** indica alta aversión al riesgo: el inversor penaliza fuertemente el riesgo y prefiere carteras más conservadoras.
- Un $\lambda$ **bajo** indica baja aversión al riesgo (alta tolerancia): el inversor está dispuesto a asumir más riesgo en busca de mayores rendimientos.
- Dos inversores con diferentes valores de $\lambda$ tendrán diferentes carteras eficientes, reflejando sus preferencias individuales.

**Solución Óptima**: Al calcular la derivada parcial e igualarla a cero (condición de primer orden), se obtiene la solución analítica:

$w^* = \frac{1}{2\lambda}\Sigma^{-1}\mu$

Esta solución muestra que los pesos óptimos dependen de la inversa de la matriz de covarianzas ($\Sigma^{-1}$), lo cual plantea desafíos computacionales cuando la matriz es mal condicionada o singular.

#### **2.2 Conceptos Iniciales de Optimización**

La optimización de carteras es un caso particular de un problema general de optimización matemática. Un problema de optimización típico tiene la forma:

$\min/\max \quad f_0(x|w)$

sujeto a: $f_i(x|w) \leq b_i$ para $i = 1,...,m$

Donde:
- **Variables**: Las cantidades que queremos determinar (en nuestro caso, los pesos $w$ de la cartera).
- **Parámetros**: Los valores conocidos o fijos (como los retornos esperados $\mu$ y la matriz de covarianzas $\Sigma$).
- **Función Objetivo**: Lo que queremos minimizar o maximizar ($f_0$).
- **Restricciones**: Las condiciones que deben cumplirse ($f_i$).

**Optimización Convexa**: Los problemas de optimización de carteras son problemas de **optimización convexa**, lo cual tiene ventajas importantes:

- **Solución Óptima Global Garantizada**: En problemas convexos, cualquier mínimo local es también un mínimo global. Esto asegura que la solución encontrada es la mejor posible.

- **Eficiencia Computacional**: Los algoritmos para resolver problemas convexos son generalmente eficientes y bien entendidos.

- **Amplia Aplicabilidad**: Muchos problemas prácticos en finanzas, ingeniería y machine learning son convexos.

La convexidad en problemas de carteras viene de:
1. **Función Objetivo Convexa**: La varianza (riesgo) es una función cuadrática de los pesos, que es convexa por definición.
2. **Restricciones Convexas**: Las restricciones comunes (suma de pesos igual a 1, pesos no negativos) son lineales y por tanto convexas.

#### **2.3 Introducción a CVXPY**

CVXPY es una librería de Python diseñada específicamente para problemas de optimización convexa. Permite formular problemas de optimización de manera intuitiva y natural, utilizando una sintaxis de alto nivel que refleja la lógica matemática del problema.

**Componentes Principales de CVXPY**:

1. **Variables de Decisión**: Representan las cantidades que queremos optimizar. Se definen como `cp.Variable(n)` donde $n$ es el número de variables.

2. **Función Objetivo**: Expresa lo que queremos minimizar o maximizar. Puede ser `cp.Minimize()` o `cp.Maximize()`.

3. **Restricciones**: Son expresiones que definen las condiciones del problema usando operadores `<=`, `>=`, `==`.

4. **Problema y Solución**: Se crea un objeto `cp.Problem()` con la función objetivo y las restricciones, y se resuelve con el método `.solve()`.

**Disciplined Convex Programming (DCP)**: CVXPY utiliza DCP para verificar automáticamente que el problema formulado es convexo. Esto garantiza que el problema es solucionable y evita errores comunes.

**Ventajas de CVXPY**:
- Sintaxis intuitiva que sigue la notación matemática.
- Integración con múltiples solvers (solucionadores) de optimización.
- Verificación automática de convexidad.
- Compatibilidad con operaciones de NumPy.

#### **2.4 Optimización de Carteras (Markowitz) - Implementación Práctica**

Con CVXPY, podemos resolver varios tipos de problemas de optimización de carteras:

**1. Máxima Rentabilidad según Tolerancia al Riesgo**: Dado un coeficiente de aversión al riesgo $\lambda$, encontrar la cartera que maximiza $w^T\mu - \lambda w^T\Sigma w$ sujeto a restricciones (como suma de pesos igual a 1, y sin posiciones cortas).

**2. Mínimo Riesgo**: Encontrar la cartera que minimiza la varianza $w^T\Sigma w$ sujeta a restricciones.

**3. Máxima Rentabilidad con Riesgo Máximo Permitido**: Maximizar $w^T\mu$ sujeto a que $w^T\Sigma w \leq \sigma_{max}^2$ (donde $\sigma_{max}$ es el nivel máximo de riesgo que el inversor está dispuesto a asumir).

**4. Mínimo Riesgo con Rentabilidad Mínima Exigida**: Minimizar $w^T\Sigma w$ sujeto a que $w^T\mu \geq \mu_{min}$ (donde $\mu_{min}$ es el rendimiento mínimo que el inversor exige).

**Restricciones Comunes**:
- **Inversión Completa**: $\sum w_i = 1$ (toda la riqueza se invierte).
- **Sin Posiciones Cortas**: $w_i \geq 0$ (no se permiten ventas en corto).
- **Límites por Activo**: $w_i \leq w_{max}$ (límite superior en la inversión en un solo activo para evitar concentración excesiva).
- **Límites Inferiores**: $w_i \geq w_{min}$ (garantizar diversificación mínima).

**La Frontera Eficiente**: Al resolver el problema para diferentes valores de $\lambda$ (o diferentes niveles de riesgo), obtenemos el conjunto de carteras eficientes, conocido como "frontera eficiente de Markowitz". Esta frontera es una curva convexa en el espacio riesgo-retorno que representa todas las carteras óptimas posibles.

---

#### **2.5 Optimización de Carteras con Posiciones Cortas**

**Definición de Posición Corta**: Una posición corta se refiere a la estrategia de venta de un activo que no se posee en ese momento, con el compromiso de recomprarlo en el futuro. El vendedor espera beneficiarse de una caída en el precio del activo.

**Mecánica de las Posiciones Cortas**:
- El inversor (tomador de posición corta) vende el activo al precio actual, recibiendo dinero.
- El broker identifica una contraparte que presta el activo.
- El inversor se compromete a devolver el activo (o su valor equivalente) en una fecha futura.
- Si el precio cae, el inversor compra el activo más barato y obtiene una ganancia.
- Si el precio sube, el inversor incurre en una pérdida.

**Implicaciones para la Optimización**:
- **Sin restricción de posiciones cortas**: Los pesos $w_i$ pueden ser negativos, lo que amplía significativamente el conjunto de carteras posibles.
- **La frontera eficiente se extiende**: Con posiciones cortas permitidas, la frontera eficiente se extiende hacia mayores rentabilidades (y también mayores riesgos), ya que permite aprovechar activos con expectativas de rendimiento negativo vendiéndolos en corto.

**Restricciones de Apalancamiento**: En la práctica, las posiciones cortas están sujetas a restricciones de apalancamiento. Una restricción común es limitar el apalancamiento total mediante:

$\sum |w_i| \leq L$

Donde $L$ es el máximo apalancamiento permitido (por ejemplo, $L = 2.0$ significa que se puede invertir hasta el doble del capital propio).

**Efectos de las Posiciones Cortas**:
- **Amplificación de Oportunidades**: Permiten aprovechar expectativas negativas sobre ciertos activos.
- **Mayor Complejidad de Riesgo**: El riesgo puede aumentar significativamente, especialmente si múltiples posiciones cortas se mueven en la misma dirección.
- **Costos Adicionales**: Las posiciones cortas típicamente involucran costos de préstamo de valores y requerimientos de margen.

La optimización con posiciones cortas requiere modificar las restricciones de no-negatividad ($w_i \geq 0$) para permitir pesos negativos, siempre que se cumplan los límites de apalancamiento.

#### **2.6 Introducción del Activo Libre de Riesgo: Teoría de Tobin**

**Características del Activo Libre de Riesgo ($R_f$)**:

El activo libre de riesgo es un instrumento financiero hipotético (o aproximadamente real, como bonos gubernamentales de países estables) que tiene características especiales:

1. **Rentabilidad Fija y Conocida**: $E(r_f) = r_f$, donde $r_f$ es una constante conocida de antemano.

2. **Varianza Cero**: $Var(r_f) = 0$, lo que significa que no hay incertidumbre sobre el rendimiento.

3. **Covarianza Cero con Otros Activos**: $Cov(r_f, r_i) = 0$ para todo activo $i$, lo que significa que el rendimiento del activo libre de riesgo no está correlacionado con ningún activo riesgoso.

**Carteras Mixtas (Activo Libre de Riesgo + Activo Riesgoso)**:

Cuando combinamos el activo libre de riesgo con un activo riesgoso en una cartera, donde $w$ es el peso en el activo libre de riesgo y $(1-w)$ es el peso en el activo riesgoso:

- **Rentabilidad de la Cartera Mixta**: $r_p = w r_f + (1-w) \mu$, donde $\mu$ es la rentabilidad esperada del activo riesgoso.

- **Volatilidad de la Cartera Mixta**: $\sigma_p = (1-w) \sigma$, donde $\sigma$ es la volatilidad del activo riesgoso.

**Resultado Fundamental**: La relación entre riesgo y rendimiento en una cartera mixta es **lineal**. Esto contrasta con la relación curva (hiperbólica) de las carteras formadas solo por activos riesgosos.

**Tipos de Inversión**:
- **Si $0 < w < 1$**: El inversor invierte parte de su capital en el activo libre de riesgo y parte en el activo riesgoso (inversión mixta conservadora).

- **Si $w = 0$**: El inversor invierte todo en el activo riesgoso.

- **Si $w = 1$**: El inversor invierte todo en el activo libre de riesgo (máxima seguridad, mínimo rendimiento).

- **Si $w < 0$**: El inversor se endeuda al tipo libre de riesgo (puede obtener $|w|$ veces su capital a la tasa $r_f$) para invertir más de su capital en el activo riesgoso (apalancamiento). Esto permite obtener mayores rendimientos esperados pero también mayores riesgos.

**Obtención de Pesos para Objetivos Específicos**:

Si tenemos una rentabilidad objetivo $r_p$, podemos despejar el peso necesario:
$w = \frac{r_p - \mu}{r_f - \mu}$

Si tenemos una volatilidad objetivo $\sigma_p$, podemos despejar:
$w = 1 - \frac{\sigma_p}{\sigma}$

**La Capital Market Line (CML) y el Teorema de Separación de Tobin**:

Cuando se introduce el activo libre de riesgo y se combina con la frontera eficiente de activos riesgosos, surge un resultado teórico fundamental:

**Teorema de Separación de Tobin**: Todos los inversores, independientemente de su aversión al riesgo, deberían mantener la misma cartera de activos riesgosos (llamada "cartera de mercado" o "cartera tangente", denotada como $M$). Las diferencias en sus preferencias de riesgo solo se reflejan en la proporción entre esta cartera de mercado y el activo libre de riesgo.

**La Capital Market Line (CML)**: Es la línea recta que conecta el activo libre de riesgo con la cartera de mercado $M$ en el espacio riesgo-retorno. La ecuación de la CML es:

$\mu_p = r_f + \left(\frac{\mu_M - r_f}{\sigma_M}\right) \sigma_p$

Donde:
- $\mu_M$ y $\sigma_M$ son la rentabilidad y volatilidad de la cartera de mercado $M$.
- El término $\frac{\mu_M - r_f}{\sigma_M}$ es el Ratio de Sharpe de la cartera de mercado, que representa la pendiente de la CML.

**Interpretación de la CML**:
- Todos los puntos en la CML son carteras eficientes cuando se puede invertir en el activo libre de riesgo.
- El segmento entre $R_f$ y $M$ representa inversiones mixtas (combinaciones de $R_f$ y $M$).
- Los puntos más allá de $M$ representan carteras apalancadas (endeudamiento en $R_f$ para invertir más en $M$).
- Ninguna cartera que esté por debajo de la CML es eficiente, ya que siempre es posible encontrar una cartera en la CML con el mismo riesgo pero mayor rendimiento, o el mismo rendimiento pero menor riesgo.

**Optimización con Activo Libre de Riesgo**:

El problema de optimización se extiende para incluir el activo libre de riesgo. La función objetivo incluye un término adicional para el activo libre de riesgo:

$\max \quad w_0 r_f + \mu^T w - \lambda w^T\Sigma w$

sujeto a: $w_0 + \sum w_i = 1$

Donde $w_0$ es el peso en el activo libre de riesgo. Esta expresión puede reescribirse usando el "premio al riesgo" $\pi = \mu - r_f$:

$\max \quad w^T\pi - \lambda w^T\Sigma w$

La solución óptima para los pesos de activos riesgosos es:

$w^* = \frac{1}{\lambda}\Sigma^{-1}\pi$

Y el peso en el activo libre de riesgo es simplemente: $w_0^* = 1 - \sum w_i^*$.

**Implicaciones Prácticas**:
- La CML proporciona una manera clara de evaluar si una cartera es eficiente.
- El teorema de separación simplifica enormemente el problema de optimización: solo necesitamos encontrar la cartera de mercado $M$ una vez, y luego cada inversor ajusta su participación según su aversión al riesgo.
- Este marco teórico es la base para modelos más avanzados como el Capital Asset Pricing Model (CAPM).

---

<a id='tema3'></a>
### **TEMA 3: MODELOS DE RIESGO SISTEMÁTICO Y VALORACIÓN DE ACTIVOS: CAPM Y MODELO DE MERCADO**

#### **3.2 Capital Asset Pricing Model (CAPM)**

El CAPM (Capital Asset Pricing Model) es un modelo de equilibrio propuesto por Sharpe, Lintner y Mossin que extiende y generaliza los conceptos del modelo de Markowitz y la teoría de Tobin. Mientras que el modelo de Markowitz sirve como guía para inversores individuales que buscan maximizar su utilidad, el CAPM se enfoca en determinar los precios de equilibrio de activos financieros arriesgados en el mercado completo.

**Supuestos del CAPM**:
- Los inversores son precio-aceptantes (ningún inversor puede manipular los precios).
- El horizonte de inversión es un período.
- Solo se puede invertir en acciones cotizadas.
- No hay costes de transacción ni impuestos.
- El inversor es racional y optimiza el ratio media-varianza.
- Solo se aborda la decisión de selección de activos para una cartera.
- La información no tiene coste y está disponible para todos los inversores.
- Las decisiones de los inversores se basan en el modelo de Markowitz.
- Expectativas homogéneas entre inversores y equilibrio de mercados de capitales.

**El Principio Fundamental del CAPM**: Según los supuestos del CAPM, todos los inversores utilizan, como activo de riesgo, la misma cartera de mercado $M$. Esto significa que la cartera tangente de la teoría de Tobin debe ser la cartera de mercado $M$. Esta cartera es eficiente y maximiza el ratio de Sharpe.

**La Fórmula del CAPM**: El CAPM establece una relación entre el premio de mercado de un activo y su riesgo sistemático:

$E[\tilde{r}_i] - r_f = \beta_i (E[\tilde{r}_M] - r_f)$

Donde:
- $E[\tilde{r}_i]$ es la rentabilidad esperada del activo $i$.
- $r_f$ es la tasa libre de riesgo.
- $E[\tilde{r}_M] - r_f$ es el premio de mercado (exceso de rentabilidad del mercado sobre la tasa libre de riesgo).
- $\beta_i$ es el coeficiente beta del activo $i$.

**El Coeficiente Beta ($\beta_i$)**: El beta mide la sensibilidad de un activo a los movimientos del mercado. Se calcula como:

$\beta_i = \frac{Cov(\tilde{r_i}, \tilde{r_M})}{\sigma^2_M}$

Donde $Cov(\tilde{r_i}, \tilde{r_M})$ es la covarianza entre el activo $i$ y el mercado $M$, y $\sigma^2_M$ es la varianza del mercado.

**Interpretación del Beta**:
- **$\beta > 1$**: El activo amplifica los movimientos del mercado. Es más volátil que el mercado y debería ofrecer mayores rentabilidades esperadas.
- **$\beta = 1$**: El activo se mueve en línea con el mercado. Su rentabilidad esperada debería igualar la del mercado.
- **$\beta < 1$**: El activo atenúa los movimientos del mercado. Es menos volátil y debería ofrecer menores rentabilidades esperadas que el mercado.
- **$\beta < 0$**: El activo se mueve en dirección opuesta al mercado (caso muy raro en la práctica).

**La Security Market Line (SML)**: Mientras que la CML (Capital Market Line) relaciona riesgo total (volatilidad) con rentabilidad, la SML relaciona riesgo sistemático (beta) con rentabilidad:

$\mu_p = r_f + \beta_p (\mu_M - r_f)$

La SML es importante porque se aplica a cualquier activo o cartera, no solo a las carteras eficientes. Todos los activos deberían estar sobre la SML en equilibrio.

**La Generalización del CAPM**: El CAPM puede derivarse formalmente considerando una cartera compuesta por un activo $i$ y la cartera de mercado $M$. Al exigir que esta cartera mixta esté en la CML cuando el peso del activo $i$ es cero, se obtiene la fórmula del CAPM. Esta derivación demuestra que el CAPM se aplica tanto a activos eficientes como no eficientes.

#### **3.3 Modelo de Mercado**

**Motivación del Modelo de Mercado**: Estimar la frontera eficiente de Markowitz requiere la matriz de varianzas-covarianzas completa. Para un mercado con 500 activos, esto significa una matriz de 500×500 = 250,000 elementos (500 varianzas + 124,750 covarianzas), lo cual es computacionalmente costoso y estadísticamente difícil de estimar con precisión. El modelo de mercado propone usar índices de mercado como proxy de la cartera de mercado y estimar el beta mediante regresión.

**La Ecuación del Modelo de Mercado**: El modelo de mercado se expresa mediante la siguiente regresión:

$R_{i,t} - R_f = \alpha_i + \beta_i (R_{M,t} - R_f) + \varepsilon_{i,t}$

O equivalentemente, usando premios al riesgo:

$\pi_{i,t} = \alpha_i + \beta_i \pi_{M,t} + \varepsilon_{i,t}$

Donde:
- $R_{i,t}$ es el rendimiento del activo $i$ en el período $t$.
- $R_{M,t}$ es el rendimiento del índice de mercado en el período $t$.
- $R_f$ es la tasa libre de riesgo.
- $\alpha_i$ es el intercepto (alpha) del activo.
- $\beta_i$ es la pendiente (beta) del activo.
- $\varepsilon_{i,t}$ es el término de error (perturbación aleatoria).

**Supuestos del Modelo de Mercado**: Para que el modelo sea válido estadísticamente, deben cumplirse los siguientes supuestos:
1. $E(\varepsilon_{i,t}) = 0$ (la esperanza del error es cero).
2. $\sigma^2(\varepsilon_{i,t}) = \sigma^2_i$ para todo $t$ (homocedasticidad: la varianza del error es constante en el tiempo).
3. $Cov(\varepsilon_{i,t}, r_{i,t}) = 0$ para todo $t$ (el error no está correlacionado con el rendimiento del activo).
4. $Cov(\varepsilon_{i,t}, \varepsilon_{i,t'}) = 0$ para $t \neq t'$ (no autocorrelación: los errores no están correlacionados entre períodos).
5. $Cov(\varepsilon_{i,t}, \varepsilon_{j,t}) = 0$ para $i \neq j$ (los errores de diferentes activos no están correlacionados).
6. $\varepsilon_{i,t} \sim N(0, \sigma^2_\varepsilon)$ (los errores siguen una distribución normal).

**Interpretación del Modelo de Mercado**:
- **$\alpha_i$ (Alpha)**: Representa la parte del rendimiento del activo que es constante en el tiempo y no depende del riesgo sistemático. En el contexto del CAPM, si $\alpha_i \neq 0$, sugiere que el activo está sobrevalorado ($\alpha_i < 0$) o subvalorado ($\alpha_i > 0$) respecto a su valor de equilibrio según el CAPM.

- **$\beta_i$ (Beta)**: Mide el riesgo sistemático del activo. Indica cuánto del rendimiento del activo está correlacionado con el rendimiento del mercado. Es la medida de sensibilidad a los movimientos del mercado.

- **$\varepsilon_{i,t}$ (Error)**: Representa el riesgo específico del activo, causado por eventos impredecibles y particulares del activo que no están relacionados con el mercado.

**Descomposición del Riesgo**: El modelo de mercado permite descomponer el riesgo total de un activo en dos componentes:

$\sigma^2(\pi_i) = \beta_i^2 \sigma^2(\pi_M) + \sigma^2(\varepsilon_i)$

Donde:
- **$\beta_i^2 \sigma^2(\pi_M)$**: Es el **riesgo sistemático** (riesgo de mercado). Este es el riesgo que no puede eliminarse mediante diversificación y por el cual el mercado ofrece compensación.

- **$\sigma^2(\varepsilon_i)$**: Es el **riesgo específico** (riesgo idiosincrático o no sistemático). Este riesgo puede eliminarse mediante diversificación y el mercado no lo compensa.

**Factores que Influyen en el Beta**:
- **Tipo de Sector**: El impacto del ciclo económico en el sector afecta el beta.
- **Diversificación de Productos**: Empresas con productos o servicios diversificados tienden a tener betas menores.
- **Diversificación Geográfica**: Puede reducir el riesgo sistemático pero introduce nuevos riesgos (tipo de cambio, riesgo país).
- **Estructura Financiera**: Mayor apalancamiento financiero (deuda) implica un beta más alto, ya que la deuda amplifica los movimientos en el valor de la empresa.

**Aplicación Práctica**: El modelo de mercado se utiliza para:
- **Estimar Betas**: Mediante regresión de mínimos cuadrados ordinarios (OLS) de los rendimientos históricos del activo sobre los rendimientos del mercado.
- **Análisis de Rendimiento**: Evaluar si un activo, cartera o fondo está generando alpha (rendimiento adicional no explicado por el riesgo sistemático).
- **Construcción de Carteras**: Seleccionar activos basándose en sus betas para construir carteras que se ajusten a diferentes estrategias de inversión (alta beta en mercados alcistas, baja beta en mercados bajistas).

**El CAPM y el Modelo de Mercado**: El modelo de mercado es una herramienta empírica para estimar los parámetros del CAPM. Si el CAPM es válido, deberíamos observar que $\alpha_i = 0$ en promedio para todos los activos. Sin embargo, la evidencia empírica muestra que muchos activos presentan alfas significativamente diferentes de cero, lo que sugiere limitaciones en el CAPM o la existencia de factores adicionales que explican los rendimientos (motivando modelos como el de Fama-French).

**Análisis por Regímenes de Mercado**: Una aplicación importante del modelo de mercado es analizar cómo cambia la relación beta-rentabilidad según el estado del mercado (alcista vs. bajista). La evidencia sugiere que:
- Las betas pueden no ser estables en el tiempo.
- La relación entre beta y rentabilidad puede variar según el régimen de mercado.
- Las estrategias de inversión basadas en beta pueden requerir ajustes dinámicos según las condiciones del mercado.

**Limitaciones y Críticas**:
- **Inestabilidad Temporal**: Las betas estimadas pueden cambiar con el tiempo, haciendo difícil su predicción.
- **Debilidad Empírica**: La evidencia empírica muestra que la relación beta-rentabilidad es más débil de lo que predice el CAPM.
- **Supuestos Irrealistas**: Muchos de los supuestos del CAPM (como ausencia de costes de transacción o expectativas homogéneas) no se cumplen en la práctica.
- **Alternativas**: Modelos como el de Fama-French han demostrado que factores adicionales (como tamaño y valor) explican mejor los rendimientos que el CAPM unifactorial.

---

**El TEMA 3 completa la teoría de valoración de activos introduciendo el CAPM como modelo de equilibrio y el modelo de mercado como herramienta empírica para estimar el riesgo sistemático y evaluar el rendimiento de activos y carteras.**

---

### **CONCLUSIONES Y CONCEPTOS CLAVE DEL MÓDULO**

El Módulo 2: Gestión de Carteras establece los fundamentos teóricos y prácticos de la teoría moderna de carteras. Los conceptos clave que emergen de este análisis son:

**1. La Diversificación Reduce el Riesgo sin Sacrificar Rendimiento**: La correlación entre activos es crucial. Cuanto más negativa sea la correlación, mayor es el beneficio de la diversificación. Sin embargo, existe un límite: el riesgo sistemático (riesgo de mercado) no puede eliminarse mediante diversificación.

**2. La Correlación Determina el Beneficio de la Diversificación**: La correlación ($\rho$) es una medida normalizada (entre -1 y 1) que indica el grado de relación lineal entre activos. Correlaciones negativas permiten una mayor reducción de riesgo mediante diversificación.

**3. La Optimización Media-Varianza Encuentra Carteras Eficientes**: El marco de Markowitz proporciona un método sistemático para encontrar carteras que maximizan el rendimiento para un nivel de riesgo dado, o minimizan el riesgo para un nivel de rendimiento dado.

**4. La Aversión al Riesgo es Subjetiva**: El coeficiente $\lambda$ refleja las preferencias individuales de cada inversor. No existe una cartera "óptima" universal; cada inversor debe encontrar su cartera óptima según su propia aversión al riesgo.

**5. La Frontera Eficiente Representa el Conjunto de Mejores Carteras**: Todas las carteras en la frontera eficiente son óptimas en el sentido de que no se puede mejorar una sin empeorar la otra (en términos de riesgo y rendimiento). La frontera es una curva convexa en el espacio riesgo-retorno.

**6. El Activo Libre de Riesgo Simplifica y Mejora las Carteras**: La introducción del activo libre de riesgo permite crear carteras con una relación lineal entre riesgo y rendimiento. El teorema de separación de Tobin muestra que todos los inversores deberían mantener la misma cartera de activos riesgosos.

**7. La Capital Market Line (CML) Define Carteras Eficientes con Activo Libre de Riesgo**: Cuando se puede invertir en el activo libre de riesgo, todas las carteras eficientes están en la CML, que conecta $R_f$ con la cartera de mercado $M$. Esta línea proporciona un benchmark claro para evaluar la eficiencia de cualquier cartera.

**8. Las Posiciones Cortas Amplían las Oportunidades pero Aumentan el Riesgo**: Permitir posiciones cortas (pesos negativos) extiende la frontera eficiente hacia mayores rendimientos, pero también hacia mayores riesgos. Requiere gestión cuidadosa del apalancamiento.

**9. La Implementación Computacional es Esencial**: Las fórmulas matriciales (como $w^T\Sigma w$) son computacionalmente eficientes para carteras con muchos activos. Herramientas como CVXPY facilitan la resolución de problemas de optimización complejos.

**10. El CAPM Extiende la Teoría a Modelos de Equilibrio**: El Capital Asset Pricing Model (CAPM) generaliza los conceptos de Markowitz y Tobin para determinar precios de equilibrio de activos. Relaciona el premio de mercado de un activo con su riesgo sistemático (beta).

**11. El Beta Mide el Riesgo Sistemático**: El coeficiente beta ($\beta_i$) cuantifica cuánto de la variabilidad de un activo está correlacionada con el mercado. Es una medida de sensibilidad a los movimientos del mercado.

**12. El Modelo de Mercado Permite Estimación Empírica**: El modelo de mercado es una herramienta práctica para estimar betas mediante regresión, evitando el cálculo costoso de matrices de covarianza completas.

**13. La Separación entre Riesgo Sistemático y Específico es Fundamental**: El modelo de mercado permite descomponer el riesgo total en riesgo sistemático (beta² × σ²M) y riesgo específico (σ²ε). Solo el riesgo sistemático es compensado por el mercado.

**14. La Security Market Line (SML) Complementa la CML**: Mientras la CML se aplica solo a carteras eficientes, la SML se aplica a cualquier activo o cartera, relacionando beta (riesgo sistemático) con rentabilidad esperada.

**15. Las Limitaciones Empíricas del CAPM Motivan Modelos Alternativos**: La evidencia muestra que factores adicionales (tamaño, valor) pueden explicar mejor los rendimientos que el CAPM unifactorial, dando lugar a modelos como el de Fama-French.

**Aplicaciones Prácticas**:

- **Construcción de Carteras Personalizadas**: Los inversores pueden usar estos métodos para construir carteras que se ajusten a su perfil de riesgo individual.

- **Evaluación de Desempeño**: La frontera eficiente y la CML proporcionan benchmarks para evaluar si una cartera está siendo gestionada eficientemente.

- **Asignación de Activos**: Las técnicas de optimización ayudan a decidir cómo distribuir el capital entre diferentes clases de activos.

- **Gestión de Riesgo**: El análisis de varianza y covarianza permite cuantificar y gestionar el riesgo total de una cartera.

- **Análisis de Eficiencia**: Los métodos presentados permiten identificar si una cartera podría mejorarse reduciendo riesgo o aumentando rendimiento.

- **Análisis de Fondos de Inversión**: Usar regresiones multifactoriales para evaluar si un fondo está generando alpha después de controlar por factores de riesgo y analizar sus exposiciones sectoriales e industriales.

- **Construcción de Carteras Multifactoriales**: Crear carteras con exposiciones objetivo a factores específicos mediante optimización Top-Down, permitiendo control directo de exposiciones a factores mientras se mantiene neutralidad a mercado.

- **Optimización Top-Down para Smart Betas**: Implementar estrategias Smart Beta que capturen primas de factores mientras mantienen control sobre riesgo y costes de transacción.

---

**Este resumen textual completo cubre todos los conceptos teóricos y prácticos del Módulo 2 hasta el TEMA 3.4 (Ejercicio Modelo de Mercado con activos del SP500), proporcionando una base sólida para entender la teoría moderna de carteras, el CAPM, y su aplicación práctica en la gestión de inversiones.**

<a id='tema4'></a>
### **TEMA 4: SMART BETAS Y MODELOS MULTIFACTORIALES**

#### **4.1 Arbitrage Pricing Theory (APT) y Modelo de Fama-French**

**Motivación de los Modelos Multifactoriales**: La evidencia empírica ha demostrado que el CAPM unifactorial tiene limitaciones significativas. Muchos activos presentan alfas significativamente diferentes de cero, sugiriendo que factores adicionales más allá del riesgo sistemático del mercado explican los rendimientos. Esto motiva el desarrollo de modelos multifactoriales como el Arbitrage Pricing Theory (APT) y el modelo de Fama-French.

**Arbitrage Pricing Theory (APT)**: El APT propuesto por Stephen Ross (1976) es un modelo más general que el CAPM. Mientras que el CAPM requiere especificar una única fuente de riesgo sistemático (el mercado), el APT permite múltiples fuentes de riesgo sistemático (factores). La idea fundamental es que los rendimientos pueden explicarse mediante múltiples factores económicos o financieros que afectan a todos los activos en el mercado.

**Modelo Multifactorial General**: Un modelo multifactorial general puede expresarse como:

$$r_{t+1} = \sum_{i=1}^t \beta_i r_i + \sum_{j=1}^n \beta_j f_j + \varepsilon_t$$

Donde el primer término captura la persistencia temporal (momentum) y el segundo término incorpora factores fundamentales que determinan la rentabilidad.

**El Modelo de Fama-French (1993)**: La principal referencia en este ámbito es Fama y French (1993), que parte del CAPM y añade tres factores adicionales para capturar el efecto tamaño y el efecto valor/crecimiento. El modelo de Fama-French extiende el CAPM unifactorial al incorporar factores adicionales que capturan anomalías empíricas en los rendimientos de los activos.

**La Ecuación del Modelo de Fama-French de 4 Factores (con Momentum)**:

$$E(r_i) = r_f + \beta_{i,MKT} E(r_m - r_f) + \beta_{i,SMB} E(SMB) + \beta_{i,HML} E(HML) + \beta_{i,MOM} E(MOM)$$

Donde:
- $r_f$ es la tasa libre de riesgo.
- $\beta_{i,MKT}$ es el beta del activo respecto al factor de mercado.
- $E(r_m - r_f)$ es el premio de mercado esperado.
- $\beta_{i,SMB}$ es el beta del activo respecto al factor tamaño (Small Minus Big).
- $E(SMB)$ es la rentabilidad esperada del factor tamaño.
- $\beta_{i,HML}$ es el beta del activo respecto al factor valor (High Minus Low).
- $E(HML)$ es la rentabilidad esperada del factor valor.
- $\beta_{i,MOM}$ es el beta del activo respecto al factor momentum.
- $E(MOM)$ es la rentabilidad esperada del factor momentum.

**Interpretación de los Factores**:

**1. Factor de Mercado (MKT)**: Es el mismo factor que en el CAPM. Representa el premio por riesgo sistemático de mercado. Los activos con mayor beta de mercado deberían ofrecer mayores rentabilidades esperadas.

**2. Factor Tamaño (SMB - Small Minus Big)**: Mide el efecto tamaño: las empresas pequeñas históricamente han ofrecido mayores rentabilidades que las grandes, controlando por beta. El factor SMB representa el diferencial de rentabilidad entre empresas pequeñas y grandes.

- **Efecto Tamaño**: Las empresas pequeñas tienen mayores costes de transacción, menor liquidez, y mayor riesgo, lo que explica teóricamente su mayor rentabilidad esperada. Sin embargo, estudios recientes cuestionan la persistencia de este efecto.

- **Análisis Temporal del Factor SMB**: Los trabajos de Dimson et al. (2011) y Fama y French (2012) cuestionan la existencia del factor tamaño a partir de 1980. El análisis empírico muestra que:
  - **Período 1927-1979**: El factor SMB muestra rentabilidades positivas significativas, con rentabilidad media anualizada positiva y ratio de Sharpe positivo.
  - **Período 1980-Hoy**: El factor SMB muestra rentabilidades significativamente menores o incluso no significativas, sugiriendo que el efecto tamaño puede haber desaparecido o reducido considerablemente.

**3. Factor Valor (HML - High Minus Low)**: Mide el efecto valor: las empresas con alto ratio book-to-market (valor) han ofrecido históricamente mayores rentabilidades que las de bajo ratio (crecimiento). El factor HML representa el diferencial de rentabilidad entre empresas "valor" y "crecimiento".

- **Efecto Valor**: Las empresas "valor" suelen estar infravaloradas o enfrentar dificultades financieras, ofreciendo mayor prima de riesgo. Este efecto ha mostrado mayor persistencia que el efecto tamaño.

- **Análisis Temporal del Factor HML**: El análisis empírico muestra que:
  - **Período 1927-2007**: El factor HML muestra rentabilidades positivas significativas, con rentabilidad media anualizada positiva y ratio de Sharpe positivo.
  - **Período 2008-Hoy**: El factor HML ha mostrado debilidad o incluso rentabilidades negativas, especialmente después de la crisis financiera de 2007-2008, sugiriendo que el efecto valor puede haber cambiado en décadas recientes.

**4. Factor Momentum (MOM)**: Mide la persistencia de los rendimientos pasados: los activos que han tenido buenos rendimientos recientes tienden a continuar haciéndolo en el corto plazo. El factor momentum captura el efecto de inercia en los precios.

- **Efecto Momentum**: Sugiere que existe inercia en los precios de los activos, posiblemente debido a comportamientos de inversores no racionales (sesgos cognitivos, reacciones lentas a información) o fricciones de mercado.

- **Análisis Temporal del Factor MOM**: El análisis empírico muestra que:
  - **Período 1927-2009**: El factor MOM muestra rentabilidades positivas significativas, aunque con alta volatilidad y períodos de pérdidas extremas (como en agosto de 1932, con pérdida mensual del 52.61%).
  - **Período 2009-Hoy**: El factor momentum ha continuado mostrando persistencia, aunque con variabilidad significativa.

**Análisis Estadístico de los Factores**:

Para validar la significatividad y estabilidad temporal de los factores, se utilizan técnicas estadísticas avanzadas que corrigen problemas comunes en series temporales financieras:

**1. Regresión HAC/Newey-West**: Los errores del modelo en series financieras suelen presentar dos problemas:
- **Heterocedasticidad**: La "intensidad del ruido" cambia con el ciclo económico. La varianza del error no es constante en el tiempo.
- **Autocorrelación**: Existe persistencia en los errores, es decir, están temporalmente correlacionados. Los errores de un período están correlacionados con los errores de períodos anteriores.

La regresión HAC (Heteroskedasticity and Autocorrelation Consistent) o Newey-West corrige estos problemas ajustando los errores estándar para que sean consistentes a pesar de la heterocedasticidad y autocorrelación. En el análisis de factores de Fama-French, típicamente se utiliza un máximo de 12 retardos mensuales (equivalente a 1 año).

**2. Rolling t-statistics**: Para evaluar la estabilidad temporal de los factores, se utilizan estadísticos t móviles (rolling t-statistics) calculados sobre ventanas deslizantes. Típicamente se utiliza una ventana de 10 años (120 meses) para datos mensuales. Esto permite observar cómo la significatividad estadística de los factores cambia con el tiempo, identificando períodos donde los factores son más o menos robustos.

**3. Análisis por Subperíodos**: Dividir la serie histórica en subperíodos (pre/post puntos de inflexión identificados empíricamente) permite evaluar la persistencia del efecto. Se calculan estadísticas descriptivas (rentabilidad media anualizada, volatilidad anualizada, ratio de Sharpe) y se realizan pruebas de significatividad estadística (t-tests con corrección HAC) para cada subperíodo.

**Estabilidad Temporal y Limitaciones de los Factores**:

Una característica importante de los factores de Fama-French es su variabilidad temporal:

- **SMB (Tamaño)**: 
  - Fuerte efecto en el período 1927-1979, pero significativamente debilitado desde 1980.
  - Los rolling t-statistics muestran que la significatividad del factor tamaño ha disminuido considerablemente en décadas recientes.
  
- **HML (Valor)**:
  - Efecto más persistente que el tamaño, con rentabilidades positivas históricamente.
  - Sin embargo, ha mostrado debilidad o incluso rentabilidades negativas en períodos posteriores a 2007 (crisis financiera).
  
- **MOM (Momentum)**:
  - Ha mostrado persistencia significativa históricamente.
  - Sin embargo, presenta alta volatilidad y períodos de pérdidas extremas (como en agosto de 1932, con pérdida mensual del 52.61%).

**Implicaciones para la Construcción de Carteras**:

La evidencia sobre la variabilidad temporal de los factores tiene implicaciones importantes para la gestión de carteras:

- **Factor-Based Investing (Smart Betas)**: Las estrategias que buscan capturar primas de factores deben considerar la estabilidad temporal de estos factores. Estrategias basadas únicamente en el factor tamaño pueden no funcionar bien en períodos recientes.

- **Selección Dinámica de Factores**: La evidencia sugiere que los factores pueden no ser estables en el tiempo. Los gestores de carteras pueden necesitar ajustar dinámicamente su exposición a factores según las condiciones del mercado y la evidencia empírica más reciente.

- **Diversificación de Factores**: En lugar de depender de un único factor, puede ser más robusto construir carteras que diversifiquen la exposición a múltiples factores, reconociendo que diferentes factores pueden ser relevantes en diferentes períodos.

**Relación con el APT (Arbitrage Pricing Theory)**:

El modelo de Fama-French puede verse como una implementación práctica del APT de Stephen Ross, que postula que los rendimientos de los activos pueden explicarse por múltiples factores de riesgo. El APT es más general que el CAPM, ya que no requiere especificar qué factores son relevantes, aunque el modelo de Fama-French propone factores específicos basados en evidencia empírica.

**Aplicación Práctica del Modelo de Fama-French**:

1. **Estimación de Rentabilidades Esperadas**: Usar los betas multifactoriales para calcular rentabilidades esperadas más precisas que con el CAPM unifactorial, incorporando los factores de tamaño, valor y momentum.

2. **Construcción de Carteras Factor-Based**: Crear carteras que se enfoquen en factores específicos (p. ej., carteras con alta exposición a SMB o HML). Estas estrategias se conocen como "Smart Betas" o "Factor-Based Investing".

3. **Análisis de Desempeño**: Evaluar si un fondo o activo está generando alpha después de controlar por factores de Fama-French. Un alpha positivo después de ajustar por factores sugiere gestión activa exitosa.

4. **Gestión de Riesgo**: Descomponer el riesgo de una cartera en componentes atribuibles a diferentes factores, permitiendo una gestión de riesgo más granular.

**Limitaciones y Consideraciones**:

- **Inestabilidad Temporal**: Los factores pueden perder significatividad en períodos futuros, como lo demuestra la evidencia empírica para SMB post-1980 y HML post-2007.

- **Sesgo de Superviviencia**: Los datos históricos pueden estar sesgados por empresas que sobrevivieron, potencialmente sobreestimando las rentabilidades históricas de ciertos factores.

- **Costes de Transacción**: Implementar estrategias factor-based puede tener costes elevados, especialmente para empresas pequeñas (factor SMB), que pueden erosionar las primas de factores.

- **Cambios Estructurales**: Los mercados evolucionan, y factores que funcionaron en el pasado pueden no funcionar en el futuro. La evidencia sobre la variabilidad temporal de los factores subraya la importancia de evaluar continuamente la relevancia de diferentes factores.

- **Modelos Alternativos**: Existen otros modelos multifactoriales, como el modelo de Fama-French de 5 factores (2015) que añade factores de rentabilidad e inversión, y modelos basados en macro-factores que incluyen variables macroeconómicas (inflación, PIB, etc.).

---

**El TEMA 4 extiende los modelos de valoración de activos al incorporar factores adicionales más allá del riesgo sistemático del mercado, permitiendo una explicación más completa de los rendimientos de los activos y proporcionando herramientas para la construcción de carteras basadas en factores (Smart Betas). Sin embargo, la evidencia empírica sobre la variabilidad temporal de los factores subraya la importancia de evaluar continuamente su relevancia y considerar estrategias de gestión dinámica de factores.**

#### **4.2 Análisis de Fondos de Inversión**

**Motivación**: Los fondos de inversión son vehículos financieros que reúnen capital de varios inversores para invertir colectivamente en una diversa cartera de activos gestionada por profesionales. Para evaluar el rendimiento y las características de estos fondos, es esencial entender qué factores explican sus rendimientos y cómo se alinean con las estrategias declaradas.

**Herramientas de Clasificación de Fondos**: Morningstar es una plataforma reconocida que ofrece análisis detallados de fondos de inversión. Una herramienta clave es el **Style Box**, que clasifica fondos según dos dimensiones:

- **Tamaño (Capitalización de Mercado)**: Pequeña, Mediana, Grande
- **Estilo**: Valor, Mezcla, Crecimiento

El Style Box proporciona una comprensión rápida y visual de la naturaleza de la inversión del fondo, permitiendo a los inversores evaluar si un fondo se alinea con sus preferencias o expectativas.

**Análisis Multifactorial de Fondos**: Para descomponer los rendimientos de un fondo y entender qué factores explican su desempeño, se utilizan regresiones multifactoriales. El análisis típico incluye:

**1. Regresión con Factores de Fama-French**: La regresión del rendimiento del fondo sobre los factores de mercado permite descomponer el rendimiento en:
- **Alpha**: Rendimiento adicional no explicado por los factores (medida de gestión activa exitosa)
- **Betas Factoriales**: Exposiciones a los factores MKT, SMB, HML, MOM
- **R^2**: Porcentaje de varianza explicada por los factores

**2. Regresión con Factores por Industria**: Para entender la exposición sectorial del fondo, se puede regresar el rendimiento del fondo sobre factores de industria (por ejemplo, sectores del GICS o factores industriales específicos). Esto permite identificar:
- **Exposiciones Sectoriales**: Qué industrias o sectores explican mejor los rendimientos del fondo
- **Concentración Sectorial**: Si el fondo está concentrado en ciertos sectores o industrias
- **Alineación con Declaraciones**: Si las exposiciones observadas coinciden con la estrategia declarada del fondo

**3. Regresión con Activos Individuales**: Para fondos que publican sus principales participaciones (por ejemplo, los 10 activos en los que más invierte), se puede regresar el rendimiento del fondo sobre los rendimientos de estos activos individuales. Esto permite:
- **Identificar Activos Clave**: Qué activos individuales explican mejor los rendimientos del fondo
- **Evaluar Concentración**: Si el fondo depende fuertemente de unos pocos activos
- **Descomponer Rendimiento**: Separar el rendimiento atribuible a activos específicos del resto

**Interpretación del Alpha**: En el contexto de modelos multifactoriales:

- **Alpha Positivo y Significativo**: Sugiere que el fondo está generando rendimientos adicionales no explicados por los factores considerados. Esto puede indicar:
  - Gestión activa exitosa (skill del gestor)
  - Exposición a factores no capturados por el modelo
  - Selección superior de activos

- **Alpha No Significativo o Negativo**: Sugiere que el fondo no está generando rendimientos adicionales después de ajustar por factores. Esto puede indicar:
  - El fondo es esencialmente una réplica factor-based (estilo Smart Beta)
  - La gestión activa no está añadiendo valor
  - El fondo puede estar incurriendo en costes que erosionan el alpha

**Aplicaciones Prácticas**:

1. **Evaluación de Desempeño**: Usar regresiones multifactoriales para evaluar si un fondo está generando alpha después de controlar por factores de riesgo.

2. **Análisis de Exposiciones**: Identificar las exposiciones reales del fondo a diferentes factores, sectores o activos, y compararlas con la estrategia declarada.

3. **Selección de Fondos**: Usar el análisis multifactorial para seleccionar fondos que se alineen con objetivos de inversión específicos (por ejemplo, fondos con alta exposición a valor o momentum).

4. **Construcción de Carteras de Fondos**: Usar análisis de factores para construir carteras de fondos diversificadas en términos de exposiciones factoriales.

**Limitaciones del Análisis**:

- **Datos Limitados**: Los fondos pueden no publicar información completa sobre sus participaciones o exposiciones sectoriales.

- **Cambios Dinámicos**: Las exposiciones de los fondos pueden cambiar con el tiempo, y el análisis histórico puede no reflejar la composición actual.

- **Modelos Incompletos**: Los modelos multifactoriales pueden no capturar todos los factores relevantes, y el alpha puede estar sesgado si faltan factores importantes.

---

#### **4.3 Diseño de Carteras con Factores: Aproximación Top-Down**

**Motivación**: La construcción de carteras basadas en factores (Smart Betas o Factor-Based Investing) busca crear carteras con exposiciones objetivo específicas a factores de riesgo, en lugar de simplemente maximizar rentabilidad esperada y minimizar varianza. La aproximación Top-Down comienza definiendo ex ante las exposiciones deseadas a factores y luego optimiza los pesos de los activos para lograr estas exposiciones.

**Aproximación Top-Down**: En contraste con la aproximación Bottom-Up (que selecciona activos individuales basándose en características y luego agrega), la aproximación Top-Down:

1. **Define Objetivos de Exposición**: Especifica ex ante las exposiciones deseadas a factores (por ejemplo, β*_value = 0.4, β*_mom = 0.3, β*_MKT = 1.0)

2. **Construye Matriz de Características**: Crea una matriz X (N × K) donde cada fila es un activo y cada columna es una característica (normalizada o beta estimado)

3. **Optimiza Pesos**: Resuelve un problema de optimización que minimiza la desviación de las exposiciones objetivo mientras controla riesgo y rotación

**Objetivos Típicos de una Cartera Multifactorial**:

- **Exposición a Factores**: Exposiciones objetivo a factores como Value, Momentum, Size
- **Neutralidad a Mercado**: Beta de cartera igual a 1 (o próxima a 1) para evitar simplemente tener más o menos beta que el mercado
- **Volatilidad Objetivo**: Volatilidad objetivo específica (por ejemplo, 12% anual)
- **Restricciones de Posiciones**: Long-only (sin posiciones cortas) o con límites de apalancamiento

**La Matriz X de Características**: La matriz X (N × K) contiene las características de los activos que determinan sus exposiciones a factores:

$$
X = \left[\begin{array}{ccccc}
z_{1,Value} & z_{1,Mom} & \beta_1 & \dots & Sector_M \\
z_{2,Value} & z_{2,Mom} & \beta_2 & \dots & Sector_M\\
\vdots & \vdots & \vdots & \dots & \vdots \\
z_{N,Value} & z_{N,Mom} & \beta_N & \dots & Sector_M
\end{array}\right]
$$

Donde:
- **$z_{i,k}$** es la medida normalizada del factor k para el activo i:
  $$z_{i,k} = \frac{señal_{i,k} - \mu_k}{\sigma_k}$$
  
  Donde $\mu_k$ y $\sigma_k$ se calculan en el corte cross-sectional del correspondiente mes. Si $z_{i,k} > 0$, el activo se considera alto en el factor k; si $z_{i,k} < 0$, se considera bajo.

- **$Sector_m$** es una variable binaria que toma valor 1 si la empresa pertenece al sector m.

**Estimation de Señales de Factores**:

**1. Señal de Momentum (MOM 12-2)**: Define el momentum como los rendimientos acumulados desde t-12 hasta t-2, excluyendo el último mes t-1 y el actual t:

$$MOM_{i,t} = \prod_{j=2}^{12}(1+r_{i,t-j}) - 1$$

Esta medida captura la persistencia de rendimientos pasados, evitando el efecto de reversión a corto plazo (mes t-1) y el efecto de ruido en el mes actual.

**2. Señal de Valor**: Se puede usar el inverso del ratio Price-to-Book (1/PB) como proxy para la señal de valor. Empresas con alto 1/PB (bajo PB) son consideradas "valor", mientras que empresas con bajo 1/PB (alto PB) son consideradas "crecimiento".

**3. Beta de Mercado**: Se estima mediante regresión del rendimiento del activo sobre el rendimiento del mercado (índice de referencia como IBEX35 o S&P500):

$$\beta_i = \frac{Cov(r_i, r_M)}{\sigma^2_M}$$

**Exposición de Cartera a Factores**: Si w es el vector de pesos de los activos (N × 1), la exposición de la cartera a los factores es:

$$b(w) = X^T w \in \mathbb{R}^K$$

Cada componente de b(w) es un promedio ponderado:

$$(X^T w)_k = \sum_{i=1}^N w_i X_{i,k}$$

Esta expresión muestra que la exposición de la cartera al factor k es simplemente la suma ponderada de las características de los activos en ese factor.

**Problema de Optimización Top-Down**: El problema de optimización tiene múltiples componentes en la función objetivo:

**1. Tracking de Exposiciones**: Minimiza la desviación de las exposiciones objetivo:

$$\|X^T w - b^*\|^2_W = (X^T w - b^*)^T W (X^T w - b^*) = \sum_{k=1}^K W_k ((X^T w)_k - b^*_k)^2$$

Donde:
- $b^*$ es el vector de exposiciones objetivo (K × 1)
- $W_k$ es el peso asociado al factor k, que determina la importancia relativa de cumplir el objetivo para ese factor

**Pesos de Exposición ($W_k$)**: Los pesos $W_k$ tienen múltiples utilidades:

- **Determinar Importancia Relativa**: $W_k$ grande significa que no se tolera desviarse del objetivo en el factor k; $W_k$ pequeño significa que el factor es secundario y se acepta desviación.

- **Normalizar por Incertidumbre**: Para normalizar factores con distintas escalas o incertidumbre:
  $$W_k = \frac{1}{\sigma^2_k}$$
  
  Donde $\sigma^2_k$ puede ser:
  - La dispersión cross-sectional de la exposición del factor: $std(X_{·,k})^2$
  - La varianza de las betas estimadas si se tienen betas de regresión

- **Determinar Tolerancia**: Para especificar tolerancia explícita al error en cada factor ($\delta_k$):
  $$W_k = \frac{1}{\delta_k^2}$$

**2. Penalización por Riesgo**: Controla el riesgo de la cartera:

$$\lambda w^T \Sigma w = \lambda \sigma^2(r_p)$$

Donde:
- $\lambda$ es el coeficiente de aversión al riesgo
- $\Sigma$ es la matriz de varianzas-covarianzas de los activos

**3. Penalización por Rotación**: Controla el coste de transacción al penalizar cambios respecto a una cartera previa:

$$\tau \|w - w^{prev}\|_2^2 = \tau \sum_i (w_i - w_i^{prev})^2$$

Donde:
- $\tau$ grande implica cambios suaves (menor rotación, menores costes de transacción, pero peor ajuste a objetivos)
- $\tau$ pequeño implica libertad para reconfigurar la cartera (mayor rotación, mayores costes, pero mejor ajuste)

**Restricciones Adicionales**:

- **Inversión Completa**: $\sum_{i=1}^N w_i = 1$ (toda la riqueza se invierte)
- **Sin Posiciones Cortas**: $w_i \geq 0$ (long-only, no se permiten ventas en corto)
- **Límites por Activo**: $w_i \leq w_{max}$ (para evitar concentración excesiva en un solo activo)
- **Neutralidad a Mercado (Opcional)**: $\beta^T w = \beta_{target}$ (beta de cartera igual al beta objetivo, típicamente 1)

**Función Objetivo Completa**: El problema de optimización Top-Down se formula como:

$$\min_w \quad \sum_{k=1}^K W_k ((X^T w)_k - b^*_k)^2 + \lambda w^T \Sigma w + \tau \|w - w^{prev}\|_2^2$$

sujeto a:
- $\sum_{i=1}^N w_i = 1$
- $w_i \geq 0$ para todo i
- $w_i \leq w_{max}$ para todo i
- $\beta^T w = \beta_{target}$ (opcional, si se desea neutralidad a mercado)

**Implementación con CVXPY**: Este problema es convexo y se puede resolver eficientemente usando CVXPY. La estructura es similar a la optimización de Markowitz, pero con un término adicional de tracking de exposiciones.

**Ventajas de la Aproximación Top-Down**:

1. **Control Directo de Exposiciones**: Permite controlar directamente las exposiciones a factores, en lugar de seleccionar activos y esperar que las exposiciones resulten correctas.

2. **Transparencia**: Las exposiciones objetivo son explícitas y comprensibles.

3. **Flexibilidad**: Permite combinar múltiples objetivos (exposiciones, riesgo, rotación) en una única optimización.

4. **Neutralidad a Mercado**: Permite mantener beta de mercado neutral mientras se tienen tilts a factores específicos.

**Desafíos y Limitaciones**:

1. **Estimación de Características**: La calidad de las exposiciones depende de la calidad de las señales de factores (momentum, valor, etc.).

2. **Estabilidad Temporal**: Las exposiciones pueden cambiar con el tiempo, requiriendo rebalanceo regular.

3. **Costes de Transacción**: El rebalanceo frecuente para mantener exposiciones objetivo puede generar costes elevados, especialmente si $\tau$ es pequeño.

4. **Complejidad Computacional**: Para grandes universos de activos (N grande) o muchos factores (K grande), el problema puede ser computacionalmente costoso.

**Aplicaciones Prácticas**:

1. **Construcción de Carteras Factor-Based**: Crear carteras con exposiciones objetivo a factores específicos (por ejemplo, alta exposición a valor, moderada a momentum).

2. **Estrategias Smart Beta**: Implementar estrategias Smart Beta que capturen primas de factores mientras mantienen neutralidad a mercado.

3. **Benchmarking Factor-Based**: Crear carteras benchmark que repliquen exposiciones factoriales específicas.

4. **Overlay de Factores**: Añadir tilts factoriales a carteras existentes mediante optimización incremental.

---

**El TEMA 4 completa la teoría y práctica de modelos multifactoriales, desde el análisis teórico de factores (4.1), pasando por la evaluación práctica de fondos de inversión (4.2), hasta la construcción sistemática de carteras basadas en factores (4.3). Estos métodos proporcionan herramientas poderosas para la gestión moderna de carteras, permitiendo a los inversores capturar primas de factores mientras mantienen control sobre riesgo y costes de transacción.**

---

<a id='conclusiones'></a>
##  CONCLUSIONES Y PRÓXIMOS PASOS

### Resumen de Conceptos Clave:

1. **Diversificación**: Reduce el riesgo sin sacrificar rendimiento (hasta el límite sistemático)
2. **Correlación**: Determina el beneficio de la diversificación (ρ = -1 óptimo, ρ = 1 sin beneficio)
3. **Optimización Media-Varianza**: Encuentra carteras eficientes que maximizan rendimiento para un nivel de riesgo dado
4. **Frontera Eficiente**: Conjunto de carteras óptimas en el espacio riesgo-retorno
5. **Activo Libre de Riesgo**: Permite crear carteras con relación lineal entre riesgo y rendimiento (CML)
6. **Teoría de Tobin**: Todos los inversores deberían mantener la misma cartera de activos riesgosos (cartera de mercado M)
7. **CAPM**: Modelo de equilibrio que relaciona premio de mercado con riesgo sistemático (beta)
8. **Beta**: Coeficiente que mide sensibilidad de activo a movimientos del mercado
9. **Modelo de Mercado**: Herramienta empírica para estimar alpha y beta mediante regresión
10. **Riesgo Sistemático vs Específico**: Solo el riesgo sistemático es compensado por el mercado
11. **Modelos Multifactoriales**: Los modelos como Fama-French extienden el CAPM incorporando factores adicionales (tamaño SMB, valor HML, momentum MOM)
12. **Análisis de Fondos**: Las regresiones multifactoriales permiten descomponer rendimientos de fondos en exposiciones a factores, sectores e industrias
13. **Carteras Multifactoriales**: La aproximación Top-Down permite construir carteras con exposiciones objetivo a factores específicos mediante optimización

### Fórmulas Más Importantes:

#### Carteras:
- **Rentabilidad**: $E(\tilde{R}_p) = \sum_{i=1}^{N}w_i E(\tilde{R}_i)$
- **Varianza**: $\sigma_p^2 = w^T\Sigma w$ (forma matricial)
- **Dos activos**: $\sigma_P^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\rho_{12}\sigma_1\sigma_2$

#### Optimización (Markowitz):
- **Función objetivo**: $\max f(w) = w^T\mu - \lambda w^T\Sigma w$
- **Solución**: $w^* = \frac{1}{2\lambda}\Sigma^{-1}\mu$

#### Tobin (CML):
- **Capital Market Line**: $\mu_p = r_f + \left(\frac{\mu_M - r_f}{\sigma_M}\right) \sigma_p$

#### CAPM (TEMA 3):
- **Fórmula CAPM**: $E[\tilde{r}_i]-r_f=\beta_i(E[\tilde{r}_M-r_f])$
- **Beta**: $\beta_i=\frac{Cov(\tilde{r_i},\tilde{r_M})}{\sigma^2_M}$
- **Security Market Line**: $\mu_p=r_f+\beta_p (\mu_M-r_f)$

#### Modelo de Mercado (TEMA 3):
- **Regresión**: $R_{i,t} - R_f = \alpha_i + \beta_i (R_{M,t} - R_f) + \varepsilon_{i,t}$
- **Descomposición Riesgo**: $\sigma^2(\pi_i)=\beta_i^2\sigma^2(\pi_M)+\sigma^2(\varepsilon_i)$

#### Modelos Multifactoriales (TEMA 4):
- **Fama-French 4 Factores**: $E(r_i) = r_f + \beta_{i,MKT} E(r_m - r_f) + \beta_{i,SMB} E(SMB) + \beta_{i,HML} E(HML) + \beta_{i,MOM} E(MOM)$
- **Momentum 12-2**: $MOM_{i,t} = \prod_{j=2}^{12}(1+r_{i,t-j})-1$
- **Exposición de Cartera**: $b(w) = X^T w \in \mathbb{R}^K$
- **Tracking de Exposiciones**: $\|X^T w - b^*\|^2_W = \sum_{k=1}^K W_k ((X^T w)_k - b^*_k)^2$

### Aplicaciones Prácticas:

* Construcción de carteras personalizadas según perfil de riesgo
* Optimización de asignación de activos
* Análisis de eficiencia de carteras
* Estimación de betas y evaluación de alpha de activos/fondos
* Análisis de riesgo sistemático vs específico
* Construcción de carteras condicionadas a regímenes de mercado (alta/baja beta)
* Análisis de fondos de inversión usando factores de Fama-French
* Construcción de carteras multifactoriales con exposiciones objetivo a factores
* Optimización Top-Down para Smart Betas y Factor-Based Investing
* Implementación con Python (CVXPY, yfinance, pandas, numpy, statsmodels)

---

**Notebook Ejecutivo creado para el Módulo 2: Gestión de Carteras**

*Última actualización: Resumen completo hasta TEMA 4.3 (Diseño de carteras con factores)*